In [1]:
'''
Import packages
'''
import numpy as np
import math
import cmath
np.set_printoptions(precision = 5)

<Token var=<ContextVar name='format_options' default={'edgeitems': 3, 'threshold': 1000, 'floatmode': 'maxprec', 'precision': 8, 'suppress': False, 'linewidth': 75, 'nanstr': 'nan', 'infstr': 'inf', 'sign': '-', 'formatter': None, 'legacy': 9223372036854775807, 'override_repr': None} at 0x105520e50> at 0x1053fd500>

# Euler's formula
$$e^{ix}=\cos x+i\sin x\longrightarrow e^{i2\pi/n}=\cos (2\pi/n) + i\sin (2\pi/n).$$

In [2]:
def complexExp(x):
    return np.cos(x) + 1j*np.sin(x)

# The nth root of unity
Given n, $\zeta_n=e^{2\pi i/n}=e^{i(2\pi /n)}$ is the nth root of unity since $\zeta_n^n=e^{in(2\pi /n)}=e^{2\pi i}=1$. 

$$e^{ix}=\cos x+i\sin x\longrightarrow e^{i(2\pi/n)}=\cos (2\pi/n) + i\sin (2\pi/n).$$

Return $(\zeta^1_n,\ldots,\zeta^{n-1}_n,1)$. 

In [3]:
def findAllRoots(n):
    first_root = complexExp(2*np.pi/n)
    return first_root ** (np.arange(n) + 1)

# Find the order of a complex number
We assume this number is the nth root of unity. We want to find the n when it is not too large (i.e., < 10000).

In [4]:
def findOrderScalar(u, m):
    i = 1
    while i < 10000:
        if(np.linalg.norm(u**i - m) < 1e-5):
            return i
        i = i + 1

# Find the order of a unitary matrix

In [5]:
def findOrderUnitary(U):
    vals = np.linalg.eigvals(U)
    orders = [findOrderScalar(e, 1) for e in vals]
    return np.lcm.reduce(orders)

# Make scalar matrix

In [6]:
def makeScalarId(dimension, w):
    # Create a diagonal matrix with scalar w as the diagonal elements
    return w * np.eye(dimension)

# Check if two unitaries are equal up to global phase
$V = \omega^t U$, where $U,V$ are two unitaries, $\omega = e^{2\pi i/3}$, $t\in \mathbb{Z}_3$. Then $V - \omega^t U = 0$.

In [7]:
def equalUpToScalar(U, V, tol=1e-10):
    """
    Check if two unitary matrices U and V are equal up to (-w)^t, 
    where w is the third root of unity and t is in Z6.

    Parameters:
    U : np.ndarray
        The first unitary matrix
    V : np.ndarray
        The second unitary matrix
    tol : float
        The numerical tolerance for equality checks (default: 1e-10).

    Returns:
    bool : True if U and V are equal up to a scalar, False otherwise.
    """
    # Compute the third root of unity w
    w = np.exp(2j * np.pi / 3)

    # Define the all-zero matrix based on the dimension and data type of the input matrix
    zero_matrix = np.zeros_like(U)
    
    # When V = U
    if np.allclose(V - U, zero_matrix, rtol=0, atol=tol):
        print("Exactly equal")
        return True
    # When V = wU
    elif np.allclose(V - w * U, zero_matrix, rtol=0, atol=tol):
        print("Equal up to w")
        return True
    # When V = w^2U
    elif np.allclose(V - (w**2) * U, zero_matrix, rtol=0, atol=tol):
        print("Equal up to w^2")
        return True
    # When V = -U
    elif np.allclose(V + U, zero_matrix, rtol=0, atol=tol):
        print("Equal up to -1")
        return True
    # When V = -wU
    elif np.allclose(V + w * U, zero_matrix, rtol=0, atol=tol):
        print("Equal up to -w")
        return True
    # When V = -w^2U
    elif np.allclose(V + (w**2) * U, zero_matrix, rtol=0, atol=tol):
        print("Equal up to -w^2")
        return True
    else:
        print("Not equal")
        return False

# Define qutrit Clifford operators

In [8]:
# The single-qutrit identity operator
id1 = np.identity(3)

# The two-qutrit identity operator
id2 = np.identity(9)

# The three-qutrit identity operator
id3 = np.identity(27)

# The third root of unity
thirdRootOfUnity = findAllRoots(3)
w = thirdRootOfUnity[0]
w_squared = thirdRootOfUnity[1]

# The single-qutrit Pauli X operator
X = np.eye(3)[[2, 0, 1]]

# The single-qutrit Pauli Z operator
Z = np.diag(w ** np.arange(3))

# The qutrit Hadamard gate
scalar = 1/(w_squared - w)
H = np.array([[1, 1, 1], [1, w, w_squared], [1, w_squared, w]]) * scalar

# The qutrit S gate
S = np.diag([w, w, 1])

# The qutrit CZ gate
CZ = np.diag([1, 1, 1, 1, w, w_squared, 1, w_squared, w])

# The qutrit SWAP gate
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)

In [9]:
w = findAllRoots(3)[0]
wI = makeScalarId(3, w)
w2 = findAllRoots(3)[1]
w2I = makeScalarId(3, w2)

A = H @ S @ H @ S @ H @ S
equalUpToScalar(np.linalg.matrix_power(A, 6), id1)
LHS = A @ X @ A.conj().T
RHS = wI @ X @ X
equalUpToScalar(LHS, RHS)

LHS = A @ Z @ A.conj().T
RHS = w2I @ Z @ Z
equalUpToScalar(LHS, RHS)

A = S @ H
equalUpToScalar(np.linalg.matrix_power(A, 6), id1)

B = np.array([[0, 0, w2], [0, 1, 0], [w, 0, 0]])
equalUpToScalar(np.linalg.matrix_power(A, 3), B)



S_GRT = np.diag([1, 1, w])
MID = Z @ X @ w2I
LHS = S_GRT @ MID @ S_GRT.conj().T
RHS = Z @ Z @ X @ wI
equalUpToScalar(LHS, RHS)

Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
equalUpToScalar(Sprime, np.diag([w, 1, w]))

C = S @ H @ H @ S
D = np.array([[1 + w, 0, 0], [0, 0, 1 + w2], [0, 1 + w2, 0]])
equalUpToScalar(C, D)


E = H @ H
F = np.array([[-1, 0, 0], [0, 0, -1], [0, -1, 0]])
equalUpToScalar(E, F)



Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


True

# Single-qutrit reduced relations: T1

In [10]:
w = findAllRoots(3)[0]
wI = makeScalarId(3, w)
w2 = findAllRoots(3)[1]
w2I = makeScalarId(3, w2)
I = np.identity(3)

# C0: (-1)^2 = 1
LHS = (-1) ** 2
RHS = 1
equalUpToScalar(LHS, RHS)

# C1: (w)^3 = 1
LHS = (w) ** 3
RHS = 1
equalUpToScalar(LHS, RHS)

# C2: (H)^4 = 1
LHS = np.linalg.matrix_power(H, 4)
RHS = I
equalUpToScalar(LHS, RHS)

# C3: (S)^3 = 1
LHS = np.linalg.matrix_power(S, 3)
RHS = I
equalUpToScalar(LHS, RHS)

# C4: (HS^2)^3 = -wI
mid = H @ S @ S
LHS = np.linalg.matrix_power(mid, 3)
RHS = (-1) * wI
equalUpToScalar(LHS, RHS)

# C5: SS' = S'S, S' = (H^2)S(H^2)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
LHS = S @ Sprime
RHS = Sprime @ S
equalUpToScalar(LHS, RHS)

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


True

# Single-qutrit derived relations: T2

In [11]:
# R1: HSH = (-w^2) X^2 S^2 H S^2
LHS = H @ S @ H
RHS = (-1) * w2I @ np.linalg.matrix_power(X, 2) @ np.linalg.matrix_power(S, 2) @ H @ np.linalg.matrix_power(S, 2)
equalUpToScalar(LHS, RHS)

# R2: H^2 S = Z^2 S H^2
LHS = H @ H @ S
RHS = np.linalg.matrix_power(Z, 2) @ S @ np.linalg.matrix_power(H, 2)
equalUpToScalar(LHS, RHS)

# R3: SZ = ZS
LHS = S @ Z
RHS = Z @ S
equalUpToScalar(LHS, RHS)

# R4: HZ^2 = XH
LHS = H @ np.linalg.matrix_power(Z, 2)
RHS = X @ H
equalUpToScalar(LHS, RHS)

# R5: X = H S^2 H^2 S H
LHS = X
RHS = H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2) @ S @ H
equalUpToScalar(LHS, RHS)

# R6: XS = (w^2)ZSX
LHS = X @ S
RHS = Z @ S @ X @ w2I
equalUpToScalar(LHS, RHS)

# R7: X^2 = H S H^2 S^2 H
LHS = np.linalg.matrix_power(X, 2)
RHS = H @ S @ np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2) @ H
equalUpToScalar(LHS, RHS)

# R8: XZ = (w^2)ZX
LHS = X @ Z
RHS = Z @ X @ w2I
equalUpToScalar(LHS, RHS)

# R9: X^3 = I
LHS = np.linalg.matrix_power(X, 3)
RHS = I
equalUpToScalar(LHS, RHS)

# R10: Z = S H^2 S^2 H^2
LHS = Z
RHS = S @ np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2)
equalUpToScalar(LHS, RHS)

# R11: Z^2 = S^2 H^2 S H^2
LHS = np.linalg.matrix_power(Z, 2)
RHS = np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
equalUpToScalar(LHS, RHS)

# R12: Z^3 = I
LHS = np.linalg.matrix_power(Z, 3)
RHS = I
equalUpToScalar(LHS, RHS)

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


True

In [12]:
w = findAllRoots(3)[0]
wI = makeScalarId(3, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(3, w2)

Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
H2 = H @ H
H3 = H @ H @ H
S2 = S @ S
Z2 = Z @ Z
X2 = X @ X

# LHS = Sprime @ S @ H @ S2 @ H2
# print(LHS)
# RHS = Z2 @ X @ H @ S @ H3 @ wI *(-1) 
# print(RHS)
# equalUpToScalar(LHS, RHS)

LHS = H @ S2 @ H3 @ S2
RHS = Z2 @ X @ S @ H @ w2I * (-1) 
equalUpToScalar(LHS, RHS)


print("\nTest 1")
LHS = Z2 @ X2 @ Z2 @ X2
RHS = Z @ X @ w2I
equalUpToScalar(LHS, RHS)

print("\nTest 2")
LHS = H @ Sprime @ Sprime @ H @ Sprime @ Sprime @ H
RHS = (-1) * Sprime @ wI
equalUpToScalar(LHS, RHS)

print("\nTest 3")
LHS = Sprime @ Sprime @ S @ H @ Sprime @ Sprime @ H @ Sprime
RHS = H @ S2 @ H @ S2 @ Sprime @ Sprime
equalUpToScalar(LHS, RHS)


Exactly equal

Test 1
Exactly equal

Test 2
Exactly equal

Test 3
Exactly equal


True

# Two-qutrit derived relations: T2

In [13]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)
# I2 is the two-qutrit identity operator
I2 = np.identity(9)

# C6: CZ^3 = I
LHS = CZ @ CZ @ CZ
RHS = I2
equalUpToScalar(LHS, RHS)

# C7: S and CZ commute
LHS = np.kron(S, I) @ CZ
RHS = CZ @ np.kron(S, I)
equalUpToScalar(LHS, RHS)

# C8: How H^2 is pushed through a CZ gate.
LHS = np.kron(H @ H, I) @ CZ
RHS = CZ @ CZ @ np.kron(H @ H, I)
equalUpToScalar(LHS, RHS)

# R14: How X is pushed through a CZ gate.
LHS = np.kron(X, I) @ CZ
RHS = CZ @ np.kron(X, Z @ Z)
equalUpToScalar(LHS, RHS)

# R16: The order of a SWAP gate is 2
LHS = SWAP @ SWAP
RHS = I2
equalUpToScalar(LHS, RHS)

# R17: CZ and SWAP commute
LHS = SWAP @ CZ
RHS = CZ @ SWAP
equalUpToScalar(LHS, RHS)

# R18: S and SWAP commute
LHS = SWAP @ np.kron(S, I)
RHS = np.kron(I, S) @ SWAP
equalUpToScalar(LHS, RHS)

# R19: H and SWAP commute
LHS = SWAP @ np.kron(H, I)
RHS = np.kron(I, H) @ SWAP
equalUpToScalar(LHS, RHS)

# R23: How S is pushed through a CX gate
LHS = CX10 @ np.kron(S, I)
RHS = np.kron(S, S @ Z @ Z) @ CZ @ CX10 @ w2I
equalUpToScalar(LHS, RHS)

# R25: How CZ is pushed through a CX gate
LHS = CX10 @ CZ
RHS = np.kron(I, Z @ Z @ S @ S) @ CZ @ CX10 @ wI
equalUpToScalar(LHS, RHS)

# R27: How H is pushed through a CZ gate
LHS = np.kron(H, I) @ CZ
RHS = wI @ np.kron(I, Z @ S @ S @ H @ H @ H) @ CZ @ CZ @ np.kron(S, H @ S @ S) @ CX10 @ SWAP @ CZ @ np.kron(H, H @ H)
equalUpToScalar(LHS, RHS)

# R28
LHS = CZ @ CZ @ np.kron(H, I) @ CZ
RHS = wI @ np.kron(H @ S @ H @ H @ H, Z @ S @ S) @ CZ @ CZ @ np.kron(H @ S @ S, I)
equalUpToScalar(LHS, RHS)

# R29
LHS = CX10 @ CZ @ SWAP @ np.kron(H, I) @ CZ
RHS = w2I @ np.kron(I, S @ H) @ CZ @ CZ @ np.kron(S @ S @ H, H @ S)
equalUpToScalar(LHS, RHS)

# R30
LHS = CX10 @ CZ @ np.kron(H @ H, H) @ CZ
RHS = np.kron(I, Z @ S) @ CX01 @ CX01 @ np.kron(I, S @ H @ H @ S) @ CX10 @ CZ @ np.kron(I, H)
equalUpToScalar(LHS, RHS)

# R34
LHS = CX10 @ CZ @ np.kron(H, H) @ CZ
RHS = (-1) * w2I @ np.kron(X @ X @ S, Z @ Z @ S @ H @ S @ S @ H) @ CZ @ CZ @ np.kron(I, H @ S) @ CX10 @ CZ @ np.kron(H @ S @ S, H @ S @ S)
equalUpToScalar(LHS, RHS)

# R35
LHS = CX10 @ CZ @ np.kron(H, H) @ CZ @ CZ
RHS = wI @ np.kron(I, Z @ Z @ S @ S @ H @ S @ S) @ CZ @ CZ @ np.kron(S @ X @ S, H @ S @ H @ H) @ CX10 @ CZ @ np.kron(H @ S, H @ S)
equalUpToScalar(LHS, RHS)

# R36
print("\n R36")
LHS = CX10 @ CX01 @ CX10 @ CX01
RHS = np.kron(H @ H, H @ H)
equalUpToScalar(LHS, RHS)

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal

 R36
Exactly equal


True

Some Other two-qutrit relations

In [20]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)
# I2 is the two-qutrit identity operator
I2 = np.identity(9)

# # R23-8
# LHS = CX10 @ np.kron(S @ S, I)
# RHS = wI @ np.kron(S @ S, Sprime @ Sprime) @ CZ @ CZ @ CX10
# equalUpToScalar(LHS, RHS)

# # R25-5
# LHS = CX10 @ CZ @ CZ
# RHS = w2I @ np.kron(I, S @ S @ Sprime @ Sprime) @ CZ @ CZ @ CX10
# equalUpToScalar(LHS, RHS)

# # R23-25
# LHS = CX10 @ CX10 @ np.kron(S @ S, I)
# RHS = wI @ np.kron(S @ S, S @ S) @ CZ @ CX10 @ CX10
# equalUpToScalar(LHS, RHS)

# # R25-6
# LHS = CX10 @ CX10 @ CZ
# RHS = w2I @ np.kron(I, S @ S @ Sprime @ Sprime) @ CZ @ CX10 @ CX10
# equalUpToScalar(LHS, RHS)

# # R23-9
# LHS = CX10 @ CX10 @ np.kron(S, I)
# RHS = w2I @ np.kron(S, S) @ CZ @ CZ @ CX10 @ CX10
# equalUpToScalar(LHS, RHS)

# # R25-7
# LHS = CX10 @ CX10 @ CZ @ CZ
# RHS = wI @ np.kron(I, S @ Sprime) @ CZ @ CZ @ CX10 @ CX10
# equalUpToScalar(LHS, RHS)

# # R23-25'
# print("\n R23-25'")
# LHS = CX01 @ CX01 @ np.kron(I, S @ S)
# RHS = wI @ np.kron(S @ S, S @ S) @ CZ @ CX01 @ CX01
# equalUpToScalar(LHS, RHS)

# # R25-5‘
# print("\n R25-5'")
# LHS = CX01 @ CZ @ CZ
# RHS = w2I @ np.kron(S @ S @ Sprime @ Sprime, I) @ CZ @ CZ @ CX01
# equalUpToScalar(LHS, RHS)

# # R23-9'
# print("\n R23-9'")
# LHS = CX01 @ CX01 @ np.kron(I, S)
# RHS = w2I @ np.kron(S, S) @ CZ @ CZ @ CX01 @ CX01
# equalUpToScalar(LHS, RHS)

# # R23-10
# print("\n R23-10")
# LHS = CX01 @ np.kron(I, Sprime)
# RHS = w2I @ np.kron(S, Sprime) @ CZ @ CX01
# equalUpToScalar(LHS, RHS)

# # R23-25-2
# print("\n R23-25-2")
# LHS = CX10 @ np.kron(H @ S @ S, H @ S @ S)
# RHS = wI @ np.kron(H @ S, H @ S @ S) @ CZ @ CX01 @ CX01
# equalUpToScalar(LHS, RHS)

# # R25-6'
# print("\n R25-6'")
# LHS = CX01 @ CX01 @ CZ
# RHS = w2I @ np.kron(S @ S @ Sprime @ Sprime, I) @ CZ @ CX01 @ CX01
# equalUpToScalar(LHS, RHS)

# # R23-8'
# print("\n R23-8'")
# LHS = CX01 @ np.kron(I, S @ S)
# RHS = wI @ np.kron(Sprime @ Sprime, S @ S) @ CZ @ CZ @ CX01
# equalUpToScalar(LHS, RHS)

# R23-11
print("\n R23-11")
LHS = CX01 @ np.kron(I, Sprime @ Sprime)
RHS = wI @ np.kron(S @ S, Sprime @ Sprime) @ CZ @ CZ @ CX01
equalUpToScalar(LHS, RHS)

# R23-12
print("\n R23-12")
LHS = CX10 @ np.kron(Sprime, I)
RHS = w2I @ np.kron(Sprime, S) @ CZ @ CX10
equalUpToScalar(LHS, RHS)


# R23-13
print("\n R23-13")
LHS = CX10 @ np.kron(Sprime @ Sprime, I)
RHS = wI @ np.kron(Sprime @ Sprime, S @ S) @ CZ @ CZ @ CX10
equalUpToScalar(LHS, RHS)

# R23-14
print("\n R23-14")
LHS = CZ @ CX10 @ np.kron(Sprime @ Sprime, I)
RHS = wI @ np.kron(Sprime @ Sprime, S @ S) @ CX10
equalUpToScalar(LHS, RHS)

# R23-15
print("\n R23-15")
LHS = CZ @ CZ
RHS = w2I @ np.kron(Sprime, S) @ CX01 @ np.kron(I, S @ S) @ CX01 @ CX01 
equalUpToScalar(LHS, RHS)

LHS = CZ @ CZ
RHS = w2I @ np.kron(S, Sprime) @ CX10 @ np.kron(S @ S, I) @ CX10 @ CX10
equalUpToScalar(LHS, RHS)

# R23-7'
print("\n R23-7'")
LHS = CX01 @ CX01 @ CZ @ CZ
RHS = wI @ np.kron(S @ Sprime, I) @ CZ @ CZ @ CX01 @ CX01 
equalUpToScalar(LHS, RHS)

# R23-16
print("\n R23-16")
LHS = CX10 @ CX10 @ np.kron(Sprime @ Sprime, I)
RHS = wI @ np.kron(Sprime @ Sprime, Sprime @ Sprime) @ CZ @ CX10 @ CX10
equalUpToScalar(LHS, RHS)

# R61
print("\n R61")
LHS = CX01 @ CX01 @ np.kron(I, S) @ CX01
RHS = CX10 @ CX10 @ np.kron(S, I) @ CX10
equalUpToScalar(LHS, RHS)

# R62
H3 = H @ H @ H
print("\n R62")
LHS = CX01 @ CX01 @ np.kron(I, H @ S @ H3) @ CX01
RHS = CX10 @ CX10 @ np.kron(H @ S @ H3, I) @ CX10
equalUpToScalar(LHS, RHS)



 R23-11
Exactly equal

 R23-12
Exactly equal

 R23-13
Exactly equal

 R23-14
Exactly equal

 R23-15
Exactly equal
Exactly equal

 R23-7'
Exactly equal

 R23-16
Exactly equal

 R61
Exactly equal

 R62
Not equal


False

In [19]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)
# I2 is the two-qutrit identity operator
I2 = np.identity(9)
H3 = H @ H @ H
Z2 = Z @ Z
X2 = X @ X

circ1 = np.kron(I, H) @ CX10 @ np.kron(S @ H, H @ S @ H3)
LHS = circ1 @ np.kron(X, I) @ circ1.conj().T
RHS = np.kron(Z, X)
equalUpToScalar(LHS, RHS)   

LHS = circ1 @ np.kron(Z, I) @ circ1.conj().T
RHS = np.kron(X2 @ Z, X) @ w2I
equalUpToScalar(LHS, RHS)  

circ1 = np.kron(I, H) @ CX10 @ np.kron(S @ H, H @ S @ H3)
LHS = circ1 @ np.kron(I, X) @ circ1.conj().T
RHS = np.kron(X, Z)
equalUpToScalar(LHS, RHS)  

circ1 = np.kron(I, H) @ CX10 @ np.kron(S @ H, H @ S @ H3)
LHS = circ1 @ np.kron(I, Z) @ circ1.conj().T
RHS = np.kron(X, X2 @ Z)
equalUpToScalar(LHS, RHS) 

circ2 = np.kron(H @ S, H3 @ S @ H) @ CX10 @ CX10 @ np.kron(I, H)
LHS = circ2 @ np.kron(X, I) @ circ2.conj().T
RHS = np.kron(X @ Z, I) @ wI
equalUpToScalar(LHS, RHS) 

LHS = circ2 @ np.kron(Z, I) @ circ2.conj().T
RHS = np.kron(X2, X @ Z)
equalUpToScalar(LHS, RHS) 

LHS = circ2 @ np.kron(I, X) @ circ2.conj().T
RHS = np.kron(I, X @ Z)
equalUpToScalar(LHS, RHS) 

LHS = circ2 @ np.kron(I, Z) @ circ2.conj().T
RHS = np.kron(X @ Z, X2) @ wI
equalUpToScalar(LHS, RHS) 

print("\nTwo-qutrit completeness")
LHS = np.kron(I, H) @ CX10 @ np.kron(S @ H, H @ S @ H3) @ CZ
RHS = np.kron(H @ S @ H2 @ Sprime, Sprime @ H3 @ S @ H2) @ CZ @ CZ @ CX01 @ CX01 @ np.kron(H @ S, I) @ CZ @ np.kron(H, I)
equalUpToScalar(LHS, RHS) 

print("\nTwo-qutrit completeness 2")
LHS = np.kron(H, H) @ CZ @ CZ @ np.kron(Sprime @ Sprime @ H3 @ S @ H2, Sprime @ Sprime @ H3 @ S @ H2)
RHS = CZ @ CZ @ np.kron(S @ H3 @ S, S @ H3 @ S)
equalUpToScalar(LHS, RHS) 

print("\nTwo-qutrit completeness 3")
C = np.kron(Sprime @ S @ H @ Sprime @ Sprime @ H, Sprime @ S @ H @ Sprime @ Sprime @ H) @ CZ
B = np.kron(H @ Sprime @ Sprime @ H @ Sprime @ Sprime @ H, H @ Sprime @ Sprime @ H @ Sprime @ Sprime @ H) @ CZ
A = np.kron(H @ S2, H @ S2)
LHS = C @ B @ A @ wI
RHS = np.kron(H @ S2, H @ S2) @ CZ @ CZ @ np.kron(H @ S2 @ H2, H @ S2 @ H2)
equalUpToScalar(LHS, RHS) 

print("\nTwo-qutrit completeness 4")
B = np.kron(Sprime @ S @ H @ Sprime @ Sprime @ H @ Sprime, Sprime @ S @ H @ Sprime @ Sprime @ H @ Sprime)
A = CZ @ CZ @ np.kron(H @ S2, H @ S2)
LHS = B @ A
RHS = np.kron(H @ S2, H @ S2) @ CZ @ CZ @ np.kron(H @ S2 @ H2, H @ S2 @ H2)
equalUpToScalar(LHS, RHS) 

print("\nTwo-qutrit completeness 5")
mid = H @ S2 @ H @ S2 @ Sprime @ Sprime
B = np.kron(mid, mid)
A = CZ @ CZ @ np.kron(H @ S2, H @ S2)
LHS = B @ A
RHS = np.kron(H @ S2, H @ S2) @ CZ @ CZ @ np.kron(H @ S2 @ H2, H @ S2 @ H2)
equalUpToScalar(LHS, RHS) 

print("\nTwo-qutrit completeness 6")
mid = H @ S @ H2 @ S @ H3 @ S2 @ H @ S @ H2
B = np.kron(mid, mid)
A = CZ @ CZ @ np.kron(H @ S2, H @ S2)
LHS = B @ A
RHS = np.kron(S2, S2) @ CZ @ CZ @ np.kron(H @ S2 @ H2, H @ S2 @ H2)
equalUpToScalar(LHS, RHS) 

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal

Two-qutrit completeness
Exactly equal

Two-qutrit completeness 2
Not equal

Two-qutrit completeness 3
Exactly equal

Two-qutrit completeness 4
Exactly equal

Two-qutrit completeness 5
Not equal

Two-qutrit completeness 6
Exactly equal


True

Some other three-qutrit relations

In [22]:
# Other three-qutrit relations
w = findAllRoots(3)[0]
wI = makeScalarId(27, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(27, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)

# CZ02 is the CZ gate acting on the first and the third qutrit
CZ02 = np.kron(I, SWAP) @ np.kron(CZ, I) @ np.kron(I, SWAP)

# CX02 is the CX gate acting on the first and the third qutrit
# The first qutrit is the control, and the third qutrit is the target
CX02 = np.kron(I, SWAP) @ np.kron(CX01, I) @ np.kron(I, SWAP)

# CX20 is the CX gate acting on the first and the third qutrit
# The first qutrit is the target, and the third qutrit is the control
CX20 = np.kron(SWAP, I) @ np.kron(I, CX10) @ np.kron(SWAP, I)
# CX20 = np.kron(H @ H @ H, I2) @ CZ02 @ np.kron(H, I2)

# I is the one-qutrit identity operator
I = np.identity(3)

# I2 is the two-qutrit identity operator
I2 = np.identity(9)

# I3 is the three-qutrit identity operator
I3 = np.identity(27)

# R43
mid = np.kron(I, CX10 @ CX10) @ CX20 @ CX20
LHS = mid @ np.kron(CZ, I)
RHS = w2I @ np.kron(CZ, Sprime @ Sprime @ S @ S) @ np.kron(I, CZ) @ CZ02 @ mid
equalUpToScalar(LHS, RHS)     

LHS = np.kron(CX10, I) @ CZ02
RHS = CZ02 @ np.kron(CX10, I) @ np.kron(I, CZ @ CZ)
equalUpToScalar(LHS, RHS)     


LHS = np.kron(I, CZ @ CZ @ np.kron(H, I) @ CZ @ CZ @ np.kron(H, I)) @ np.kron(CZ, I) @ np.kron(I, np.kron(H, I) @ CZ @ CZ @ np.kron(H, I))
RHS = np.kron(I, np.kron(H, I) @ CZ @ CZ @ np.kron(H, I)) @ np.kron(CZ, I) @ np.kron(I, np.kron(H, I) @ CZ @ CZ @ np.kron(H, I) @ CZ @ CZ)
equalUpToScalar(LHS, RHS)   

LHS = np.kron(I, CX10 @ CX10 @ CZ @ CZ) @ np.kron(CX01 @ CX01, I) @ np.kron(I,CZ)
RHS = np.kron(I, CZ @ CZ) @ np.kron(CX01 @ CX01, I) @ np.kron(I, CZ @ CX10 @ CX10)
equalUpToScalar(LHS, RHS) 

# C16-7'
# print("\n C16-7'")
# LHS = CX20 @ CX20 @ np.kron(CZ @ CZ, I)
# RHS = np.kron(I, CZ @ CZ) @ np.kron(CZ @ CZ, I) @ CX20 @ CX20
# equalUpToScalar(LHS, RHS)

# # C16-2'
# print("\n C16-2'")
# LHS = np.kron(I, CX10 @ CX10) @ np.kron(CZ @ CZ, I)
# RHS = CZ02 @ CZ02 @ np.kron(CZ @ CZ, I) @ np.kron(I, CX10 @ CX10)
# equalUpToScalar(LHS, RHS)

# # R47
# print("\n R47-int")
# IHI = np.kron(I, np.kron(H, I))
# LHS = np.kron(CX10 @ CZ, I) @ CZ02 @ IHI @ np.kron(I, CZ)
# #RHS = np.kron(I, np.kron(H @ H @ H, I)) @ np.kron(I, CZ @ CZ @ CX10 @ CX10) @ IHI @ np.kron(CX10 @ CZ, I) @ IHI @ CX20 @ CX20
# RHS = np.kron(I, CX10 @ CX10 @ CZ) @ np.kron(CX10 @ CZ, I) @ IHI @ CX20 @ CX20
# equalUpToScalar(LHS, RHS) 

# # C16-13
# print("\n C16-13")
# LHS = np.kron(I, CX10) @ np.kron(CZ @ CZ, I)
# RHS = np.kron(CZ @ CZ, I) @ np.kron(I, CX10) @ CZ02
# equalUpToScalar(LHS, RHS) 

# # C16-14
# print("\n C16-14")
# LHS = np.kron(I, CX10) @ np.kron(CX10 @ CX10, I)
# RHS = np.kron(CX10 @ CX10, I) @ np.kron(I, CX10) @ CX20
# equalUpToScalar(LHS, RHS) 

# # C16-15
# print("\n C16-15")
# LHS = np.kron(CZ @ CZ, I) @ np.kron(I, CX10)
# RHS = np.kron(I, CX10) @ np.kron(CZ @ CZ, I) @ CZ02 @ CZ02 
# equalUpToScalar(LHS, RHS) 

# # C16-8'
# print("\n C16-8'")
# LHS = np.kron(CZ @ CZ, I) @ CX20 @ CX20
# RHS = np.kron(I, CZ) @ CX20 @ CX20 @ np.kron(CZ @ CZ, I)
# equalUpToScalar(LHS, RHS) 

# # C16-3'
# print("\n C16-3'")
# LHS = np.kron(CZ @ CZ, I) @ np.kron(I, CX10 @ CX10)
# RHS = CZ02 @ np.kron(I, CX10 @ CX10) @ np.kron(CZ @ CZ, I)
# equalUpToScalar(LHS, RHS) 

# # C16-16
# print("\n C16-16")
# LHS = np.kron(CZ @ CZ, I) @ CX20
# RHS = CX20 @ np.kron(I, CZ @ CZ) @ np.kron(CZ @ CZ, I)
# equalUpToScalar(LHS, RHS) 

# # C16-17
# print("\n C16-17")
# LHS = np.kron(I, CX10 @ CX10) @ np.kron(CX10 @ CX10, I)
# RHS = np.kron(CX10 @ CX10, I) @ np.kron(I, CX10 @ CX10) @ CX20 @ CX20
# equalUpToScalar(LHS, RHS) 

# # C16-18
# print("\n C16-18")
# LHS = CX20 @ np.kron(CZ @ CZ, I)
# RHS = np.kron(CZ @ CZ, I) @ CX20 @ np.kron(I, CZ)
# equalUpToScalar(LHS, RHS) 

# # C16-19
# print("\n C16-19")
# LHS = CZ02 @ CZ02 @ np.kron(I, CX01)
# RHS = np.kron(I, CX01) @ np.kron(CZ @ CZ, I) @ CZ02 @ CZ02
# equalUpToScalar(LHS, RHS) 

# # C16-20
# print("\n C16-20")
# LHS = CX20 @ np.kron(I, CX01)
# RHS = np.kron(I, CX01) @ CX20 @ np.kron(CX10, I)
# equalUpToScalar(LHS, RHS) 

# # C13-12
# print("\n C13-12")
# LHS = np.kron(I, CX10) @ np.kron(CX01, I)
# RHS = np.kron(CX01, I) @ np.kron(I, CX10)
# equalUpToScalar(LHS, RHS) 

# # C13-13
# print("\n C13-13")
# LHS = CX02 @ np.kron(I, CX01)
# RHS = np.kron(I, CX01) @ CX02
# equalUpToScalar(LHS, RHS) 

# # C13-14
# print("\n C13-14")
# LHS = CX20 @ np.kron(CX10, I)
# RHS = np.kron(CX10, I) @ CX20
# equalUpToScalar(LHS, RHS) 

# C16-21
# print("\n C16-21")
# LHS = CX20 @ CX20 @ np.kron(I, CX01)
# RHS = np.kron(I, CX01) @ np.kron(CX10 @ CX10, I) @ CX20 @ CX20
# equalUpToScalar(LHS, RHS) 

# # C16-22
# print("\n C16-22")
# LHS = CZ02 @ CZ02 @ np.kron(I, CX01 @ CX01)
# RHS = np.kron(CZ, I) @ np.kron(I, CX01 @ CX01) @ CZ02 @ CZ02
# equalUpToScalar(LHS, RHS) 

# # C16-23
# print("\n C16-23")
# LHS = CX20 @ CX20 @ np.kron(CZ, I)
# RHS = np.kron(CZ, I) @ np.kron(I, CZ) @ CX20 @ CX20
# equalUpToScalar(LHS, RHS) 

# # C16-24
# print("\n C16-24")
# LHS = CX20 @ CX20 @ np.kron(I, CX01 @ CX01)
# RHS = np.kron(CX10, I) @ np.kron(I, CX01 @ CX01) @ CX20 @ CX20
# equalUpToScalar(LHS, RHS) 

# # C16-25
# print("\n C16-25")
# LHS = np.kron(CX10 @ CX10, I) @ np.kron(I, CX10 @ CX10)
# RHS = np.kron(I, CX10 @ CX10) @ np.kron(CX10 @ CX10, I) @ CX20
# equalUpToScalar(LHS, RHS) 

# # C16-26
# print("\n C16-26")
# LHS = np.kron(CX10 @ CX10, I) @ np.kron(I, CX10)
# RHS = np.kron(I, CX10) @ np.kron(CX10 @ CX10, I) @ CX20 @ CX20
# equalUpToScalar(LHS, RHS) 

# # Test
# print("\nTest")
# fixed = np.kron(CX10 @ CZ, I) @ CX20 @ CZ02
# LHS = np.kron(np.kron(Z @ X, I), I) @ fixed
# RHS = fixed @ np.kron(np.kron(Z @ X, I), I)
# equalUpToScalar(LHS, RHS) 

# # C16-27
# print("\n C16-27")
# LHS = np.kron(I, CX10 @ CX10) @ np.kron(CZ, I)
# RHS = np.kron(CZ, I) @ np.kron(I, CX10 @ CX10) @ CZ02
# equalUpToScalar(LHS, RHS) 

# # C16-28
# print("\n C16-28")
# LHS = np.kron(CX01, I) @ np.kron(I, CZ @ CZ)
# RHS = np.kron(I, CZ @ CZ) @ np.kron(CX01, I) @ CZ02
# equalUpToScalar(LHS, RHS) 

# # C16-29
# print("\n C16-29")
# LHS = CX02 @ np.kron(I, CZ @ CZ)
# RHS = np.kron(I, CZ @ CZ) @ CX02 @ np.kron(CZ, I)
# equalUpToScalar(LHS, RHS) 

# # C16-30
# print("\n C16-30")
# LHS = np.kron(CX01 @ CX01, I) @ np.kron(I, CZ)
# RHS = np.kron(I, CZ) @ np.kron(CX01 @ CX01, I) @ CZ02
# equalUpToScalar(LHS, RHS)

# # C16-31
# print("\n C16-31")
# LHS = CZ02 @ np.kron(I, CX01 @ CX01)
# RHS = np.kron(I, CX01 @ CX01) @ CZ02 @ np.kron(CZ @ CZ, I)
# equalUpToScalar(LHS, RHS) 

# # C16-32
# print("\n C16-32")
# LHS = np.kron(CZ, I) @ np.kron(I, CX10 @ CX10)
# RHS = np.kron(I, CX10 @ CX10) @ np.kron(CZ, I) @ CZ02 @ CZ02
# equalUpToScalar(LHS, RHS) 

# # C16-33
# print("\n C16-33")
# LHS = np.kron(I, CZ) @ np.kron(CX01 @ CX01, I)
# RHS = np.kron(CX01 @ CX01, I) @ np.kron(I, CZ) @ CZ02 @ CZ02
# equalUpToScalar(LHS, RHS)

# C16-34
print("\n C16-34")
LHS = np.kron(CX10, I) @ np.kron(I, CX10)
RHS = np.kron(I, CX10) @ np.kron(CX10, I) @ CX20
equalUpToScalar(LHS, RHS)

# C16-35
print("\n C16-35")
LHS = np.kron(I, CX10) @ np.kron(CX10, I)
RHS = np.kron(CX10, I) @ np.kron(I, CX10) @ CX20 @ CX20
equalUpToScalar(LHS, RHS)

# C16-36
print("\n C16-36")
LHS = np.kron(CX01 @ CX01, I) @ np.kron(I, CZ)
RHS = np.kron(I, CZ) @ np.kron(CX01 @ CX01, I) @ CZ02
equalUpToScalar(LHS, RHS)

Exactly equal
Exactly equal
Exactly equal
Exactly equal

 C16-34
Exactly equal

 C16-35
Exactly equal

 C16-36
Exactly equal


True

In [44]:
# Other three-qutrit relations: dynamic testing
w = findAllRoots(3)[0]
wI = makeScalarId(27, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(27, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)

# CZ02 is the CZ gate acting on the first and the third qutrit
CZ02 = np.kron(I, SWAP) @ np.kron(CZ, I) @ np.kron(I, SWAP)

# CX02 is the CX gate acting on the first and the third qutrit
# The first qutrit is the control, and the third qutrit is the target
CX02 = np.kron(I, SWAP) @ np.kron(CX01, I) @ np.kron(I, SWAP)

# CX20 is the CX gate acting on the first and the third qutrit
# The first qutrit is the target, and the third qutrit is the control
CX20 = np.kron(SWAP, I) @ np.kron(I, CX10) @ np.kron(SWAP, I)
# CX20 = np.kron(H @ H @ H, I2) @ CZ02 @ np.kron(H, I2)

# I is the one-qutrit identity operator
I = np.identity(3)

# I2 is the two-qutrit identity operator
I2 = np.identity(9)

# I3 is the three-qutrit identity operator
I3 = np.identity(27)
H2 = H @ H
H3 = H @ H @ H
S2 = S @ S




 C16-34
Exactly equal

 C16-35
Exactly equal


True

In [15]:
# Three-qutrit relations: intermediary steps
w = findAllRoots(3)[0]
wI = makeScalarId(27, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(27, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)

# CZ02 is the CZ gate acting on the first and the third qutrit
CZ02 = np.kron(I, SWAP) @ np.kron(CZ, I) @ np.kron(I, SWAP)

# CX02 is the CX gate acting on the first and the third qutrit
# The first qutrit is the control, and the third qutrit is the target
CX02 = np.kron(I, SWAP) @ np.kron(CX01, I) @ np.kron(I, SWAP)

# CX20 is the CX gate acting on the first and the third qutrit
# The first qutrit is the target, and the third qutrit is the control
CX20 = np.kron(SWAP, I) @ np.kron(I, CX10) @ np.kron(SWAP, I)
# CX20 = np.kron(H @ H @ H, I2) @ CZ02 @ np.kron(H, I2)

# I is the one-qutrit identity operator
I = np.identity(3)

# I2 is the two-qutrit identity operator
I2 = np.identity(9)

# I3 is the three-qutrit identity operator
I3 = np.identity(27)

# R59
# print("\nR59")
# H2 = H @ H
# H3 = H @ H @ H
# S2 = S @ S
# LHS = np.kron(CX10 @ CZ, I) @ CX20 @ CZ02 @ np.kron(I, np.kron(H, H))
#wire1 = S2 @ Sprime @ H @ S @ H2 @ S2 @ H
# wire1 = Z @ Z @ X @ X
# wire2 = S @ Sprime @ Sprime @ H3 @ S
# wire3 = S @ H3
# tail = np.kron(wire1, np.kron(wire2, wire3) @ CZ @ CZ @ CX10)
# mid = np.kron(CX10, H) @ np.kron(I, CZ) @ np.kron(np.kron(I, I), S2)
# RHS = (-1) * w2I @ tail @ mid @ CX20 @ np.kron(CZ, I) @ CZ02 @ np.kron(I, CX10 @ np.kron(H, H @ S2) @ CX01 @ CZ)
# equalUpToScalar(LHS, RHS)

# # R59
# print("\nR59")
# H2 = H @ H
# H3 = H @ H @ H
# S2 = S @ S
# IHI = np.kron(I, np.kron(H, I))
# LHS = np.kron(CX10, I) @ CX20 @ np.kron(I, CZ)
# #wire1 = S2 @ Sprime @ H @ S @ H2 @ S2 @ H
# mid = CZ @ CZ @ CX01
# component3 = np.kron(X @ X, np.kron(S @ Sprime @ Sprime @ H, I) @ mid @ np.kron(H @ Sprime @ Sprime, I) @ mid) @ IHI
# component2 = np.kron(CX10, I) @ CX20 @ np.kron(I, CZ)
# wire1 = Sprime @ S2 @ H @ S2 @ H2
# component1 = np.kron(I, np.kron(wire1, wire1) @ CZ @ CZ @ np.kron(H, H))
# RHS = (-1) * component3 @ component2 @ component1
# equalUpToScalar(LHS, RHS)

# w = findAllRoots(3)[0]
# wI = makeScalarId(9, w)


# LHS = np.kron(S @ Sprime @ Sprime @ H3 @ S, S @ H3) @ CZ @ CZ @ CX10 @ np.kron(I, H) @ CZ @ np.kron(I, S2)
# RHS = w2I @ np.kron(S @ Sprime @ Sprime @ H, I) @ CZ @ CZ @ CX01 @ np.kron(H @ Sprime @ Sprime, I) @ CZ @ CZ @ CX01 @ np.kron(H, I) @ CZ
# equalUpToScalar(LHS, RHS)

# R59
# print("\nR59")
# H2 = H @ H
# H3 = H @ H @ H
# S2 = S @ S
# LHS = np.kron(CX10, I) @ CX20 @ np.kron(I, CZ)
# #wire1 = S2 @ Sprime @ H @ S @ H2 @ S2 @ H
# component3 = np.kron(X @ X, np.kron(S @ Sprime @ Sprime @ H @ Sprime, S) @ CX01 @ np.kron(H, I) @ CX01 @ np.kron(H, S2))
# component2 = np.kron(CX10, I) @ CX20 @ np.kron(I, CZ)
# wire1 = Sprime @ S2 @ H @ S2 @ H2
# component1 = np.kron(I, np.kron(wire1, wire1) @ CZ @ CZ @ np.kron(H, H))
# RHS = (-1) * wI @ component3 @ component2 @ component1
# equalUpToScalar(LHS, RHS)


# H2 = H @ H
# H3 = H @ H @ H
# S2 = S @ S
# LHS = np.kron(S @ Sprime @ Sprime @ H @ Sprime, S) @ CX01 @ np.kron(H, I) @ CX01 @ np.kron(H, I)
# RHS = np.kron(S @ Sprime @ Sprime @ H @ Sprime @ H2, S @ H3) @ CZ @ CZ @ CX10 @ np.kron(I, H)
# equalUpToScalar(LHS, RHS)

# R59
# print("\nR59")
# H2 = H @ H
# H3 = H @ H @ H
# S2 = S @ S
# LHS = np.kron(CX10, I) @ CX20 @ np.kron(I, CZ)
# component3 = np.kron(X @ X, np.kron(S @ H @ S @ H3, S @ H3 @ Sprime) @ CX10)
# component2 = np.kron(CX10, H) @ CX20 @ np.kron(I, CZ)
# wire1 = Sprime @ S @ H @ S2 @ H2
# component1 = np.kron(I, np.kron(wire1, wire1) @ CZ @ CZ @ np.kron(H, H))
# RHS = wI @ component3 @ component2 @ component1
# equalUpToScalar(LHS, RHS)

H2 = H @ H
H3 = H @ H @ H
S2 = S @ S
Z2 = Z @ Z
# component3 = np.kron(I, np.kron(I, H3)) @ np.kron(X, CX10 @ CX10 @ np.kron(H @ S2 @ H3 @ S2, Sprime @ Sprime @ H @ S2))
# LHS = w2I @ component3 @ np.kron(CX10, I) @ CX20 @ np.kron(I, CZ)

# LHS = np.kron(I, np.kron(H @ S2 @ H2 @ S2, H @ S2 @ H3 @ S2)) @ np.kron(S, CZ @ CX01 @ CX01 @ np.kron(H @ S2, I)) @ np.kron(CZ, I) @ np.kron(I, CZ)
# IHI = np.kron(I, np.kron(H, I))
# component4 = np.kron(I, CZ) @ CZ02 @ CZ02 @ np.kron(I, CX01 @ CX01) @ IHI
# component3 = np.kron(I, np.kron(I, H @ S)) @ np.kron(I, CX01 @ CX01 @ CZ) @ IHI
# component2 = np.kron(CZ @ CZ @ CX01 @ CX01, I) @ np.kron(I, CZ @ CX10 @ CX10)
# component1 = np.kron(I, np.kron(S @ H @ S2 @ H3 @ S2 @ H2, S @ Sprime @ H3))
# RHS = wI @ component1 @ component2 @ component3 @ component4
# equalUpToScalar(LHS, RHS)

IIH3 = np.kron(I, np.kron(I, H3))
IIH = np.kron(I, np.kron(I, H))
# LHS = IIH @ np.kron(I, CX10) @ np.kron(I, np.kron(S @ H, H @ S @ H3)) @ np.kron(CX10, I) @ CX20 @ np.kron(I, CZ)
# RHS = np.kron(CX10, I) @ CX20 @ np.kron(I, CZ) @ np.kron(I, np.kron(H @ S, H3 @ S @ H)) @ np.kron(I, CX10 @ CX10) @ IIH
# equalUpToScalar(LHS, RHS)

LHS = IIH @ np.kron(I, CX10) @ np.kron(I, np.kron(S @ H, H @ S @ H3)) @ np.kron(CZ, I) @ CZ02 @ np.kron(I, CZ)
RHS = wI @ np.kron(Sprime, np.kron(H @ S, Sprime)) @ np.kron(CZ, I) @ np.kron(I, CZ) @ np.kron(CX01, I) @ np.kron(I, CX10) @ np.kron(I, np.kron(I, H3 @ S @ H)) @ np.kron(I, CX10 @ CX10) @ IIH
equalUpToScalar(LHS, RHS)

Exactly equal


True

# Three-qutrit derived relations: T3

In [23]:
# Three-qutrit relations
w = findAllRoots(3)[0]
wI = makeScalarId(27, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(27, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)

# CZ02 is the CZ gate acting on the first and the third qutrit
CZ02 = np.kron(I, SWAP) @ np.kron(CZ, I) @ np.kron(I, SWAP)

# CX02 is the CX gate acting on the first and the third qutrit
# The first qutrit is the control, and the third qutrit is the target
CX02 = np.kron(I, SWAP) @ np.kron(CX01, I) @ np.kron(I, SWAP)

# CX20 is the CX gate acting on the first and the third qutrit
# The first qutrit is the target, and the third qutrit is the control
CX20 = np.kron(SWAP, I) @ np.kron(I, CX10) @ np.kron(SWAP, I)
# CX20 = np.kron(H @ H @ H, I2) @ CZ02 @ np.kron(H, I2)

# I is the one-qutrit identity operator
I = np.identity(3)

# I2 is the two-qutrit identity operator
I2 = np.identity(9)

# I3 is the three-qutrit identity operator
I3 = np.identity(27)

# R39
LHS = np.kron(I, CZ @ CZ @ np.kron(H, I)) @ np.kron(CZ, I)
RHS = np.kron(CX01 @ CX01, I) @ np.kron(I, CZ @ CZ @ np.kron(H, I)) @ np.kron(I, SWAP) @ np.kron(CZ, I) @ np.kron(I, SWAP)
equalUpToScalar(LHS, RHS)

# R40
LHS = np.kron(I, CZ @ CZ @ np.kron(H, I)) @ np.kron(CZ @ CZ, I)
RHS = np.kron(CX01, I) @ np.kron(I, CZ @ CZ @ np.kron(H, I)) @ np.kron(I, SWAP) @ np.kron(CZ @ CZ, I) @ np.kron(I, SWAP)
equalUpToScalar(LHS, RHS)

# R41
LHS = CZ02 @ CZ02 @ np.kron(H, I2) @ np.kron(CZ, I)
RHS = np.kron(CX10 @ CX10, I) @ np.kron(I, CZ) @ CZ02 @ CZ02 @ np.kron(H, I2)
equalUpToScalar(LHS, RHS)

# R42
LHS = CZ02 @ CZ02 @ np.kron(H, I2) @ np.kron(CZ @ CZ, I)
RHS = np.kron(CX10, I) @ np.kron(I, CZ @ CZ) @ CZ02 @ CZ02 @ np.kron(H, I2)
equalUpToScalar(LHS, RHS)

# R43
LHS = np.kron(I, CX10 @ CX10) @ CX20 @ CX20 @ np.kron(CZ, I)
RHS = np.kron(CZ, I) @ np.kron(S, np.kron(S, Sprime @ Sprime @ S)) @ np.kron(I, CX10 @ CX10) @ CX20 @ CX20 @ np.kron(S @ S, np.kron(S @ S, I))
equalUpToScalar(LHS, RHS)     

# R44
LHS = np.kron(I, CX10 @ CX10) @ CX20 @ CX20 @ np.kron(CZ @ CZ, I)
RHS = np.kron(CZ @ CZ, I) @ np.kron(S @ S, np.kron(S @ S, Sprime @ S @ S)) @ np.kron(I, CX10 @ CX10) @ CX20 @ CX20 @ np.kron(S, np.kron(S, I))
equalUpToScalar(LHS, RHS)  

# R45
LHS = CZ02
IHI = np.kron(I, np.kron(H, I))
RHS = IHI @ np.kron(I, CZ @ CZ) @ IHI @ np.kron(CZ, I) @ IHI @ np.kron(I, CZ @ CZ) @ IHI @ np.kron(CZ, I)
equalUpToScalar(LHS, RHS)  

# R46
LHS = np.kron(CX10, I) @ CZ02
IHI = np.kron(I, np.kron(H, I))
RHS = np.kron(I, CZ @ CZ) @ IHI @ np.kron(I, CZ @ CZ) @ IHI @ np.kron(CZ, I) @ IHI @ np.kron(I, CZ @ CZ) @ IHI @ np.kron(CZ @ CX10, I)
equalUpToScalar(LHS, RHS) 

# R47
IHI = np.kron(I, np.kron(H, I))
LHS = np.kron(CX10 @ CZ, I) @ CZ02 @ IHI @ np.kron(I, CZ)
RHS = wI @ np.kron(I, np.kron(H @ H @ H @ S @ S, I)) @ np.kron(I, CX10 @ CX10) @ np.kron(I, np.kron(S @ H, S @ S)) @ np.kron(CX10 @ CZ, I) @ IHI @ CX20 @ CX20
equalUpToScalar(LHS, RHS) 

# R48
LHS = np.kron(CX10 @ CZ, I) @ IHI @ np.kron(I, CZ @ CZ)
RHS = wI @ np.kron(I, np.kron(H @ S @ S, I)) @ np.kron(I, CX10 @ CX10) @ np.kron(I, np.kron(S @ H @ H @ H, S @ S)) @ np.kron(CX10 @ CZ, I) @ IHI @ CX20 @ CZ02
equalUpToScalar(LHS, RHS) 

# R49
LHS = np.kron(CX10 @ CZ, I) @ IHI @ CX20 @ CZ02 @ np.kron(I, CZ)
ISI = np.kron(I, np.kron(S, I))
RHS = w2I @ ISI @ np.kron(I, CX10 @ CX10) @ np.kron(I, np.kron(S @ S, S)) @ np.kron(CX10 @ CZ, I) @ IHI
equalUpToScalar(LHS, RHS) 

# R50
LHS = np.kron(CX10 @ CZ, I) @ IHI @ CX20 @ np.kron(I, np.kron(I, H @ H)) @ np.kron(I, CZ @ CZ)
RHS = w2I @ np.kron(I, np.kron(S, S @ H @ H)) @ np.kron(I, CX10) @ np.kron(I, np.kron(S @ S, I)) @ np.kron(CX10 @ CZ, I) @ IHI @ CZ02
equalUpToScalar(LHS, RHS) 

# R51
H2 = H @ H
H3 = H @ H @ H
S2 = S @ S
wire2 = H @ S2 @ H2 @ S @ H @ Sprime @ H @ S
wire3 = S @ H2 @ S2
LHS = np.kron(CX10 @ CZ, I) @ IHI @ CX20 @ CZ02 @ CZ02 @ np.kron(I, CZ @ CZ)
RHS = (-1) * wI @ np.kron(I, np.kron(wire2, wire3)) @ np.kron(I, CZ) @ np.kron(I, np.kron(H @ S @ H3, I)) @ np.kron(CX10 @ CZ, I) @ CX20 @ np.kron(I, np.kron(H, H @ H))
equalUpToScalar(LHS, RHS) 

# R52
LHS = np.kron(CX10 @ CZ, I) @ CX20 @ np.kron(np.kron(I, H), H @ H) @ np.kron(I, CZ)
RHS = (-1) * wI @ np.kron(I, np.kron(wire2, wire3)) @ np.kron(I, CZ) @ np.kron(I, np.kron(H @ S @ H3, I)) @ np.kron(CX10 @ CZ, I) @ CX20 @ CZ02 @ CZ02 @ IHI
equalUpToScalar(LHS, RHS) 

# R53
IIH = np.kron(I, np.kron(I, H))
LHS = np.kron(CZ, I) @ CX20 @ CZ02 @ IIH @ np.kron(I, CZ)
RHS = w2I @ np.kron(I, np.kron(I, S)) @ np.kron(I, CX01 @ CX01) @ np.kron(I, np.kron(S, S @ S)) @ np.kron(CX10 @ CX10, I) @ CX20 @ CZ02 @ IIH
equalUpToScalar(LHS, RHS) 

# R54
IIS = np.kron(I, np.kron(I, S))
LHS = np.kron(CX10 @ CZ @ CZ, I) @ CX20 @ CZ02 @ IIH @ np.kron(I, CZ)
RHS = IIS @ np.kron(I, CX01 @ CX01) @ np.kron(I, np.kron(S @ S @ Sprime, S @ S)) @ np.kron(CZ, I) @ CX20 @ CZ02 @ IIH
equalUpToScalar(LHS, RHS) 

# R55
LHS = np.kron(CX10, I) @ CX20 @ CZ02 @ np.kron(I, np.kron(H @ H, H)) @ np.kron(I, CZ)
RHS = wI @ np.kron(I, np.kron(Sprime @ Sprime, Sprime @ H)) @ np.kron(I, CZ @ CZ) @ np.kron(I, np.kron(H @ H, H @ S @ S @ H @ H)) @ np.kron(CX10 @ CZ @ CZ, I) @ CX20 @ CZ02 @ IIH
equalUpToScalar(LHS, RHS) 

# R56
LHS = np.kron(CZ, I) @ CX20 @ CZ02 @ IIH @ np.kron(I, CZ @ CZ)
RHS = np.kron(I, np.kron(I, Sprime @ Sprime @ S2 @ H)) @ np.kron(I, CZ @ CZ) @ np.kron(CX10 @ CZ @ CZ, H @ S @ H2 @ S) @ CX20 @ CZ02 @ IIH
equalUpToScalar(LHS, RHS)

# R57
LHS = np.kron(CX10 @ CZ @ CZ, I) @ CX20 @ CZ02 @ IIH @ np.kron(I, CZ @ CZ)
RHS = w2I @ np.kron(I, np.kron(I, Sprime @ H) @ CZ @ CZ @ np.kron(S @ H2, H @ S2 @ H2)) @ np.kron(CX10, I) @ CX20 @ CZ02 @ np.kron(I, np.kron(H2, H))
equalUpToScalar(LHS, RHS)

# R58
LHS = np.kron(CX10, I) @ CX20 @ CZ02 @ np.kron(I, np.kron(H2, H)) @ np.kron(I, CZ @ CZ)
RHS = wI @ np.kron(I, np.kron(S @ Sprime @ Sprime, S) @ CX01 @ CX01 @ np.kron(S2 @ H2, S2)) @ np.kron(CZ, I) @ CX20 @ CZ02 @ IIH
equalUpToScalar(LHS, RHS)

# R59
print("\nR59")
H2 = H @ H
H3 = H @ H @ H
S2 = S @ S
LHS = np.kron(CX10 @ CZ, I) @ CX20 @ CZ02 @ np.kron(I, np.kron(H, H) @ CZ)
#wire1 = S2 @ Sprime @ H @ S @ H2 @ S2 @ H
wire1 = Z @ Z @ X @ X
wire2 = S @ Sprime @ Sprime @ H @ Sprime @ Sprime @ H
RHS = wI @ np.kron(wire1, np.kron(wire2, wire2) @ CZ @ np.kron(H @ S2, H @ S2)) @ np.kron(CX10 @ CZ, I) @ CX20 @ CZ02 @ np.kron(I, np.kron(H @ S2, H @ S2))
equalUpToScalar(LHS, RHS)

# R60
LHS = np.kron(CX10 @ CZ, I) @ CX20 @ CZ02 @ np.kron(I, np.kron(H, H) @ CZ @ CZ)
wire1 = S @ Sprime @ Sprime @ H @ S2 @ H2 @ S @ H
wire2 = Sprime @ H @ S2
wire3 = H @ S2 @ H2
RHS = np.kron(wire1, np.kron(wire2, wire2) @ CZ @ CZ @ np.kron(wire3, wire3)) @ np.kron(CX10 @ CZ, I) @ CX20 @ CZ02 @ np.kron(I, np.kron(H @ S, H @ S))
equalUpToScalar(LHS, RHS)

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal

R59
Exactly equal
Exactly equal


True

# Three-qutrit derived relations: T2

In [97]:
# Three-qutrit relations
w = findAllRoots(3)[0]
wI = makeScalarId(27, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(27, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)

# CZ02 is the CZ gate acting on the first and the third qutrit
CZ02 = np.kron(I, SWAP) @ np.kron(CZ, I) @ np.kron(I, SWAP)

# CX02 is the CX gate acting on the first and the third qutrit
# The first qutrit is the control, and the third qutrit is the target
CX02 = np.kron(I, SWAP) @ np.kron(CX01, I) @ np.kron(I, SWAP)

# CX20 is the CX gate acting on the first and the third qutrit
# The first qutrit is the target, and the third qutrit is the control
CX20 = np.kron(SWAP, I) @ np.kron(I, CX10) @ np.kron(SWAP, I)
# CX20 = np.kron(H @ H @ H, I2) @ CZ02 @ np.kron(H, I2)

# I is the one-qutrit identity operator
I = np.identity(3)

# I2 is the two-qutrit identity operator
I2 = np.identity(9)

# I3 is the three-qutrit identity operator
I3 = np.identity(27)

# C16
LHS = np.kron(I, CZ) @ np.kron(CX01, I)
RHS = np.kron(CX01, I) @ np.kron(I, CZ) @ CZ02
equalUpToScalar(LHS, RHS)

# 

Exactly equal


True

In [13]:
# Seemingly related relations
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)
# I2 is the two-qutrit identity operator
I2 = np.identity(9)


# R23^3: How S is pushed through a CX gate
LHS = np.kron(S @ S, Sprime @ Sprime) @ CX10 @ np.kron(S, I) @ wI
RHS = CZ @ CX10
equalUpToScalar(LHS, RHS)

# R25^2: How CZ is pushed through a CX gate
LHS = np.kron(I, Sprime @ Sprime @ S @ S) @ CX10 @ CZ @ w2I
RHS = CZ @ CX10
equalUpToScalar(LHS, RHS)

Exactly equal
Exactly equal


True

In [14]:
# LHS = np.kron(I, H @ H) @ CX01 @ CX01 @ CX10 @ CX01 @ CX01
# RHS = SWAP
# equalUpToScalar(LHS, RHS)

LHS = CX10 @ CX10 @ CX01 @ CX10 @ CX10 @ np.kron(I, H @ H)
RHS = SWAP
equalUpToScalar(LHS, RHS)

Exactly equal


True

In [15]:
I = np.identity(3)
LHS = np.kron(H @ H, I) @ CZ
RHS = CZ @ CZ @ np.kron(H @ H, I)
equalUpToScalar(LHS, RHS)

Exactly equal


True

# Relations for two-qutrit Clifford operators: $R_{23}$
$$
\begin{align*}
CX_{10} &\coloneqq (H^3 \otimes I)\cdot CZ \cdot (H \otimes I)\\
S' &\coloneqq H^2 \cdot S \cdot H^2\\
R_{23}&: CX_{10}\cdot (S \otimes I) = \omega^2 \cdot (S \otimes S Z^2 )\cdot CZ \cdot CX_{10}\\
R_{23}^1&: CX_{10}\cdot (S \otimes I)  = \omega^2 \cdot (S \otimes S')\cdot CZ \cdot CX_{10}\\
R_{23}^2&: (S \otimes I) \cdot CX_{10} = \omega^2 \cdot (S^2 \otimes S')\cdot CZ \cdot CX_{10} \cdot (S^2 \otimes I)\\
R_{23}^3&: \omega \cdot (S^2 \otimes S'^2) \cdot CX_{10} \cdot (S \otimes I) = CZ \cdot CX_{10}
\end{align*}
$$

In [16]:
w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
CX = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)

# Relation 23
LHS = CX @ np.kron(S, I)
RHS = np.kron(S, S @ Z @ Z) @ CZ @ CX @ w2I
equalUpToScalar(LHS, RHS)

# Relation 23, variant 1
LHS = CX @ np.kron(S, I)
RHS = np.kron(S, Sprime) @ CZ @ CX @ w2I
equalUpToScalar(LHS, RHS)

# Relation 23, variant 2
LHS = np.kron(S, I) @ CX
RHS = np.kron(S @ S, Sprime) @ CZ @ CX @ np.kron(S @ S, I) @ w2I
equalUpToScalar(LHS, RHS)

# Relation 23, variant 3
LHS = np.kron(S @ S, Sprime @ Sprime) @ CX @ np.kron(S, I) @ wI
RHS = CZ @ CX
equalUpToScalar(LHS, RHS)

Exactly equal
Exactly equal
Exactly equal
Exactly equal


True

# Relations for two-qutrit Clifford operators: $R_{25}$
$$
\begin{align*}
CX_{10} &\coloneqq (H^3 \otimes I)\cdot CZ \cdot (H \otimes I)\\
S' &\coloneqq H^2 \cdot S \cdot H^2\\
R_{25}: CX_{10}\cdot CZ &= \omega \cdot (I \otimes Z^2 S^2)\cdot CZ \cdot CX_{10}\\
R_{25}^1: CX_{10}\cdot CZ  &= \omega \cdot (I \otimes S S')\cdot CZ \cdot CX_{10}
\end{align*}
$$

In [17]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)
CX = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)

# Relation 25
LHS = CX @ CZ
RHS = np.kron(I, Z @ Z @ S @ S) @ CZ @ CX @ wI
equalUpToScalar(LHS, RHS)

# Relation 25, variant 1
LHS = CX @ CZ
RHS = np.kron(I, S @ Sprime) @ CZ @ CX @ wI
equalUpToScalar(LHS, RHS)


Exactly equal
Exactly equal


True

# Relations for two-qutrit Clifford operators: $R_{27}$
$$
\begin{align*}
CX_{10} &\coloneqq (H^3 \otimes I)\cdot CZ \cdot (H \otimes I)\\
CX_{01} &\coloneqq (I \otimes H^3)\cdot CZ \cdot (I \otimes H)\\
S' &\coloneqq H^2 \cdot S \cdot H^2\\
SWAP &\coloneqq CZ \cdot (H \otimes H) \cdot CZ \cdot (H \otimes H) \cdot CZ \cdot (H \otimes H)\\
R_{27}&: (H \otimes I)\cdot CZ = \omega \cdot (I \otimes ZS^2H^3)\cdot CZ^2 \cdot (S \otimes HS^2) \cdot CX_{10} \cdot SWAP \cdot CZ \cdot (H \otimes H^2)\\
R_{27}^1&: (H \otimes I)\cdot CZ^2 = \omega \cdot (I \otimes S^2H)\cdot CZ^2 \cdot (S \otimes HS^2) \cdot CX_{10} \cdot SWAP \cdot CZ \cdot (H \otimes I)\\
R_{27}^2&: (H \otimes I)\cdot CZ = \omega \cdot (I \otimes ZS^2)\cdot CX_{01}^2 \cdot (S \otimes S^2) \cdot CX_{10} \cdot SWAP \cdot CZ \cdot (H \otimes H^2)
\end{align*}
$$

In [18]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)
CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

# Relation 27
LHS = np.kron(H, I) @ CZ
RHS = np.kron(I, Z @ S @ S @ H @ H @ H) @ CZ @ CZ @ np.kron(S, H @ S @ S) @ CX10 @ SWAP @ CZ @ np.kron(H, H @ H) @ wI
equalUpToScalar(LHS, RHS)

# Relation 27, variant 1
LHS = np.kron(H, I) @ CZ @ CZ
RHS = np.kron(I, S @ S @ H) @ CZ @ CZ @ np.kron(S, H @ S @ S) @ CX10 @ SWAP @ CZ @ np.kron(H, I) @  wI
equalUpToScalar(LHS, RHS)

# Relation 27, variant 2
LHS = np.kron(H, I) @ CZ
RHS = np.kron(I, Z @ S @ S) @ CX01 @ CX01 @ np.kron(S, S @ S) @ CX10 @ SWAP @ CZ @ np.kron(H, H @ H) @  wI
equalUpToScalar(LHS, RHS)

Exactly equal
Exactly equal
Exactly equal


True

# Relations for two-qutrit Clifford operators: $R_{28}$
$$
\begin{align*}
CX_{10} &\coloneqq (H^3 \otimes I)\cdot CZ \cdot (H \otimes I)\\
CX_{01} &\coloneqq (I \otimes H^3)\cdot CZ \cdot (I \otimes H)\\
S' &\coloneqq H^2 \cdot S \cdot H^2\\
SWAP &\coloneqq CZ \cdot (H \otimes H) \cdot CZ \cdot (H \otimes H) \cdot CZ \cdot (H \otimes H)\\
R_{28}&: CZ^2 \cdot (H \otimes I)\cdot CZ = \omega \cdot (HSH^3 \otimes ZS^2)\cdot CZ^2 \cdot (HS^2 \otimes I)\\
R_{28}^1 &: CZ^2 \cdot (H \otimes I)\cdot CZ = \omega \cdot (HSH^3 \otimes S'^2)\cdot CZ^2 \cdot (HS^2 \otimes I)\\
R_{28}^2 &: CZ^2 \cdot (H \otimes I)\cdot CZ = \omega \cdot (HS \otimes S'^2)\cdot  CX_{10}^2 \cdot (S^2 \otimes I)
\end{align*}
$$

In [19]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)
CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

# Relation 28
LHS = CZ @ CZ @ np.kron(H, I) @ CZ
RHS = np.kron(H @ S @ H @ H @ H, Z @ S @ S) @ CZ @ CZ @ np.kron(H @ S @ S, I) @ wI
equalUpToScalar(LHS, RHS)

# Relation 28, variant 1
LHS = CZ @ CZ @ np.kron(H, I) @ CZ
RHS = np.kron(H @ S @ H @ H @ H, Sprime @ Sprime) @ CZ @ CZ @ np.kron(H @ S @ S, I) @ wI
equalUpToScalar(LHS, RHS)

# Relation 28, variant 2
LHS = CZ @ CZ @ np.kron(H, I) @ CZ
RHS = np.kron(H @ S, Sprime @ Sprime) @ CX10 @ CX10 @ np.kron(S @ S, I) @ wI
equalUpToScalar(LHS, RHS)

Exactly equal
Exactly equal
Exactly equal


True

# Relations for two-qutrit Clifford operators: $R_{29}$
$$
\begin{align*}
CX_{10} &\coloneqq (H^3 \otimes I)\cdot CZ \cdot (H \otimes I)\\
CX_{01} &\coloneqq (I \otimes H^3)\cdot CZ \cdot (I \otimes H)\\
S' &\coloneqq H^2 \cdot S \cdot H^2\\
SWAP &\coloneqq CZ \cdot (H \otimes H) \cdot CZ \cdot (H \otimes H) \cdot CZ \cdot (H \otimes H)\\
R_{29}&: CX_{10}\cdot CZ \cdot SWAP \cdot (H \otimes I)\cdot CZ = \omega^2 \cdot (I \otimes SH)\cdot CZ^2 \cdot (S^2H \otimes HS)
\end{align*}
$$

In [20]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

# Relation 29
LHS = CX10 @ CZ @ SWAP @ np.kron(H, I) @ CZ
RHS = np.kron(I, S @ H) @ CZ @ CZ @ np.kron(S @ S @ H, H @ S) @ w2I
equalUpToScalar(LHS, RHS)

Exactly equal


True

# Relations for two-qutrit Clifford operators: $R_{30}$
$$
\begin{align*}
CX_{10} &\coloneqq (H^3 \otimes I)\cdot CZ \cdot (H \otimes I)\\
CX_{01} &\coloneqq (I \otimes H^3)\cdot CZ \cdot (I \otimes H)\\
S' &\coloneqq H^2 \cdot S \cdot H^2\\
SWAP &\coloneqq CZ \cdot (H \otimes H) \cdot CZ \cdot (H \otimes H) \cdot CZ \cdot (H \otimes H)\\
R_{30}&: CX_{10}\cdot CZ \cdot SWAP \cdot (H \otimes I)\cdot CZ^2 \\
&=(I \otimes ZS)\cdot CX_{01}^2 \cdot (I \otimes SH^2S) \cdot CX_{10} \cdot CZ \cdot SWAP \cdot (H \otimes H^2)\\
R_{30}^1&: (H^3 \otimes I)\cdot CZ \cdot (H \otimes I)\cdot CZ \cdot SWAP \cdot (H \otimes I)\cdot CZ^2 \\
&=(I \otimes ZSH^3)\cdot CZ^2 \cdot (H^3 \otimes HSH^2S) \cdot CZ \cdot (H \otimes I)\cdot SWAP \cdot CZ \cdot (H \otimes H^2)\\
\end{align*}
$$

In [21]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

# Relation 30
LHS = CX10 @ CZ @ SWAP @ np.kron(H, I) @ CZ @ CZ
RHS = np.kron(I, Z @ S) @ CX01 @ CX01 @ np.kron(I, S @ H @ H @ S) @ CX10 @ CZ @ SWAP @ np.kron(H, H @ H)
equalUpToScalar(LHS, RHS)

# Relation 30, variant 1
LHS = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I) @ CZ @ SWAP @ np.kron(H, I) @ CZ @ CZ
RHS = np.kron(I, Z @ S @ H @ H @ H) @ CZ @ CZ @ np.kron(H @ H @ H, H @ S @ H @ H @ S) @ CZ @ np.kron(H, I) @ SWAP @ CZ @ np.kron(H, H @ H)

equalUpToScalar(LHS, RHS)

Exactly equal
Exactly equal


True

In [22]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

LHS = CX10 @ np.kron(I, H @ H) @ CX10 @ CX10
RHS = np.kron(I, H @ H) @ CX10
equalUpToScalar(LHS, RHS)

Exactly equal


True

In [23]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

 # Relation 34
LHS = CX10 @ CZ @ np.kron(H, H) @ CZ
RHS = (-1) * w2I @ np.kron(X @ X @ S, Z @ Z @ S @ H @ S @ S @ H) @ CZ @ CZ @ np.kron(I, H @ S) @ CX10 @ CZ @ np.kron(H @ S @ S, H @ S @ S)
equalUpToScalar(LHS, RHS)

Exactly equal


True

In [24]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

 # Relation 35
LHS = CX10 @ CZ @ np.kron(H, H) @ CZ @ CZ
RHS = wI @ np.kron(I, Z @ Z @ S @ S @ H @ S @ S) @ CZ @ CZ @ np.kron(S @ X @ S, H @ S @ H @ H) @ CX10 @ CZ @ np.kron(H @ S, H @ S)
equalUpToScalar(LHS, RHS)

Exactly equal


True

In [25]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

LHS = np.kron(I, S @ H @ H @ S) @ CZ @ CZ @ CX10 @ CX10 @ CX01 @ CX01
RHS = np.kron(H @ H, S @ H @ H @ S @ H @ H) @ CZ @ CZ @ CX01 @ CX10
equalUpToScalar(LHS, RHS)

LHS = CX10 @ CX10 @ CX01 @ CX01
RHS = np.kron(H @ H, H @ H) @ CX01 @ CX10
equalUpToScalar(LHS, RHS)

Exactly equal
Exactly equal


True

In [26]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

LHS = np.identity(9)
S2 = S @ S
H2 = H @ H
H3 = H @ H @ H
top = S @ H @ S2 @ H2 @ S @ H3 @ S2 @ H3 @ S @ H2 @ S2 @ H
bottom = H @ S @ H2 @ S @ H3 @ S @ H2 @ S @ H3 @ Sprime @ S
RHS = np.kron(top, bottom) * (-1)
equalUpToScalar(LHS, RHS)
# LHS = CZ @ np.kron(H, I)
# top = S @ H @ S @ S @ H @ H @ S @ H @ Sprime @ Sprime
# bottom = H @ S @ H @ H @ S @ H @ H @ H @ S @ H @ H @ S @ H @ H @ H
# RHS = (-1) * np.kron(top, bottom) @ CZ @ np.kron(H @ S @ Sprime @ Sprime, S @ S)
# equalUpToScalar(LHS, RHS)
# LHS = CX10 @ CZ @ np.kron(H, H) @ CZ @ CZ
# top = S @ H @ S @ S @ H @ H @ S @ H @ Sprime @ Sprime
# bottom = S @ Sprime @ H @ S @ H @ H @ S @ H @ H @ H @ S @ H @ H @ S @ H @ H @ H
# RHS5 = (-1) * wI @ np.kron(top, bottom) @ CZ @ np.kron(H @ S @ Sprime @ Sprime, S @ S @ H) @ CX01 @ CX01 @ CZ @ CZ
# top = S @ H @ S @ S @ H @ H @ S @ H @ Sprime @ Sprime
# bottom = S @ Sprime @ H @ S @ H @ H @ S @ H @ H @ H @ S @ H @ H @ S
# RHS4 = wI @ np.kron(top, bottom) @ CX01 @ np.kron(H, H @ H) @ CZ @ np.kron(S, H @ H @ Sprime @ H @ S)
# top = S @ H @ S @ S @ H @ H @ S @ H @ Sprime @ Sprime
# bottom = S @ Sprime @ H @ S @ H @ H @ S @ H @ H @ H @ S @ H @ H @ S
# RHS3 = wI @ np.kron(top, bottom) @ CX01 @ CX10 @ np.kron(H @ S, Sprime @ H @ S)
# RHS2 = w2I @ np.kron(S @ H @ S @ S @ H @ H @ S @ H @ S, 
#                      S @ Sprime @ H @ S @ H @ H @ S @ H @ H @ H @ S @ H @ H) @ CZ @ CX01 @ CX10 @ np.kron(S @ H @ S, S @ H @ S)
# RHS1 = wI @ np.kron(S @ H @ S @ S @ H @ H @ S @ H @ S, 
#                     S @ Sprime @ H @ S @ Sprime) @ CZ @ CZ @ np.kron(I, H @ S @ H @ H) @ CX10 @ CZ @ np.kron(H @ S, H @ S)
# RHS = np.kron(S @ H @ S @ S @ H @ H @ S @ H @ Sprime @ Sprime @ S, S @ Sprime @ H @ S @ H @ H @ S @ H @ H @ H @ S @ S @ Sprime @ H @ H) @ CX01 @ CX10 @ np.kron(H @ S, H @ S)
# RHS = wI @ np.kron(I, Z @ Z @ S @ S @ H @ S @ S) @ CZ @ CZ @ np.kron(S @ X @ S, H @ S @ H @ H) @ CX10 @ CZ @ np.kron(H @ S, H @ S)
# equalUpToScalar(LHS, RHS4)

# equalUpToScalar(LHS, RHS)

Exactly equal


True

In [27]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

LHS = np.kron(Sprime, S) @ CX01 @ np.kron(H, S @ S) @ CZ @ np.kron(H @ H @ S, H @ S @ S) @ CX10
RHS = np.kron(H, H) @ CZ @ np.kron(H @ S, H)
equalUpToScalar(LHS, RHS)

# LHS = wI @ CZ @ np.kron(H, H) @ CZ @ np.kron(H @ H @ S, H @ S @ S) @ CX10
# RHS = np.kron(I, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H @ S, H)
# equalUpToScalar(LHS, RHS)

Exactly equal


True

In [28]:
w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

LHS = CX10 @ CX10 @ np.kron(H @ H, I)
RHS = SWAP
equalUpToScalar(LHS, RHS)

Not equal


False

In [225]:


w = findAllRoots(3)[0]
wI = makeScalarId(9, w)

w2 = findAllRoots(3)[1]
w2I = makeScalarId(9, w2)

CX10 = np.kron(H @ H @ H, I) @ CZ @ np.kron(H, I)
CX01 = np.kron(I, H @ H @ H) @ CZ @ np.kron(I, H)
Sprime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
SWAP = CZ @ np.kron(H, H) @ CZ @ np.kron(H, H) @ CZ @ np.kron(H, H)
I = np.identity(3)

LHS = np.kron(I, S @ Sprime) @ wI
RHS = CX10 @ np.kron(S, I) @ CX10 @ np.kron(S, I) @ CX10 @ np.kron(S, I)
equalUpToScalar(LHS, RHS)


LHS = np.kron(S, S @ S) @ CX10 @ CZ @ wI
RHS = CX10
equalUpToScalar(LHS, RHS)

Exactly equal
Not equal


False

In [29]:
def testScalar():
    # The third root of unity
    thirdRootOfUnity = findAllRoots(3)
    w = thirdRootOfUnity[0]
    w_squared = thirdRootOfUnity[1]

    print("w = ", w)
    print("The order of w is ",findOrderScalar(w, 1))
    print("w^2 = ", w_squared)
    print("The order of w^2 is ",findOrderScalar(w_squared, 1))
    print("\n")

In [30]:
testScalar()

w =  (-0.49999999999999983+0.8660254037844387j)
The order of w is  3
w^2 =  (-0.5000000000000002-0.8660254037844384j)
The order of w^2 is  3




In [31]:
w = findAllRoots(3)[0]
wI = makeScalarId(3, w)
I = np.identity(3)

newS = H.conj().T @ S @ H
LHS = S @ newS
RHS = newS @ S
equalUpToScalar(LHS, RHS)

# newH = H @ H

# mid = newH @ newH @ newS @ newH @ newH
# LHS = newS @ mid
# RHS = mid @ newS

# LHS = np.linalg.matrix_power(newH @ newS @ newS, 3)
# RHS = (-1) * wI
# equalUpToScalar(LHS, RHS)




Not equal


False

In [32]:
w = findAllRoots(3)[0]
wI = makeScalarId(3, w)
w2 = findAllRoots(3)[1]
w2I = makeScalarId(3, w2)
id1 = np.identity(3)

# LHS = (-1) * w2I
# S_prime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2) 
# RHS = np.linalg.matrix_power(H, 3) @ Z @ H @ np.linalg.matrix_power(S_prime, 2) 
# equalUpToScalar(LHS, RHS)

# I = np.identity(3)

print("w", w)
print("w2", w2)


# LHS = I
# med = np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3)
# RHS = np.linalg.matrix_power(med, 6)

# print(RHS)
# equalUpToScalar(LHS, RHS)

LHS = (-1) * wI
S_prime = np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2) 
RHS = S_prime @ S
print("S'S", RHS)
equalUpToScalar(LHS, RHS)
RHS = S @ S_prime
print("SS'", RHS)
equalUpToScalar(LHS, RHS)

LHS = np.diag([w2, w, w])
equalUpToScalar(LHS, RHS)

RHS = S
LHS = S_prime @ Z
equalUpToScalar(LHS, RHS)

# LHS = (-1) * I
# med = np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3)
# RHS = np.linalg.matrix_power(med, 3)

# testLHS = np.array([[0, 0, w], [0, 1, 0], [w2, 0, 0]])

# print(testLHS @ testLHS)

# print("w", w)
# print("w2", w2)
# print(RHS)
# equalUpToScalar(testLHS, RHS)


w (-0.49999999999999983+0.8660254037844387j)
w2 (-0.5000000000000002-0.8660254037844384j)
S'S [[-5.00000e-01-8.66025e-01j -2.17916e-17-9.92914e-17j
  -1.73451e-16-9.67400e-17j]
 [-2.10550e-17-1.00567e-16j -5.00000e-01+8.66025e-01j
  -3.31999e-17+1.88002e-16j]
 [ 1.71241e-16-1.00567e-16j -1.46215e-16-1.22753e-16j
  -5.00000e-01+8.66025e-01j]]
Not equal
SS' [[-5.00000e-01-8.66025e-01j -2.17916e-17-9.92914e-17j
   1.70505e-16-1.01843e-16j]
 [-2.10550e-17-1.00567e-16j -5.00000e-01+8.66025e-01j
  -1.46215e-16-1.22753e-16j]
 [-1.72714e-16-9.80157e-17j -3.31999e-17+1.88002e-16j
  -5.00000e-01+8.66025e-01j]]
Not equal
Exactly equal
Exactly equal


True

# Single-qutirt Reduced Relations


In [33]:
w = findAllRoots(3)[0]
wI = makeScalarId(3, w)
w2 = findAllRoots(3)[1]
negwI = (-1) * wI
w2I = makeScalarId(3, w2)

# (HS^2)^3 = -wI   
equalUpToScalar(np.linalg.matrix_power(H @ S @ S, 3), negwI) 

# S(H^2)S(H^2) = (H^2)S(H^2)S
H2 = np.linalg.matrix_power(H, 2)
equalUpToScalar(S @ H2 @ S @ H2, H2 @ S @ H2 @ S)


Exactly equal
Exactly equal


True

In [34]:
def testGate(G):
    print(np.matrix(G))
    print("The order of the gate is ",findOrderUnitary(G))
    print("\n")

In [35]:
testGate(S)

[[-0.5+0.86603j  0. +0.j       0. +0.j     ]
 [ 0. +0.j      -0.5+0.86603j  0. +0.j     ]
 [ 0. +0.j       0. +0.j       1. +0.j     ]]
The order of the gate is  3




In [36]:
def testHGate():
    thirdRootOfUnity = findAllRoots(3)
    w = thirdRootOfUnity[0]
    w_squared = thirdRootOfUnity[1]
    scalar = 1/(w_squared - w)
    
    print("1/(w^2 - w) = ", scalar)
    print("w/(w^2 - w) = ", w * scalar)
    print("w^2/(w^2 - w) = ", w_squared * scalar)
    print("\n")
    print(np.matrix(H))
    print("The order of the gate is ",findOrderUnitary(H))
    print("\n")

In [37]:
testHGate()

1/(w^2 - w) =  (-1.2952601953960163e-16+0.5773502691896258j)
w/(w^2 - w) =  (-0.5-0.2886751345948129j)
w^2/(w^2 - w) =  (0.49999999999999994-0.2886751345948129j)


[[-1.29526e-16+0.57735j -1.29526e-16+0.57735j -1.29526e-16+0.57735j]
 [-1.29526e-16+0.57735j -5.00000e-01-0.28868j  5.00000e-01-0.28868j]
 [-1.29526e-16+0.57735j  5.00000e-01-0.28868j -5.00000e-01-0.28868j]]
The order of the gate is  4




# Pauli anticommutation: X gate
- $XZX^\dagger = \omega^2 Z \iff XZ = \omega^2 ZX$
- $XXX^\dagger = X$
- $X^\dagger Z X = \omega Z \iff ZX = \omega XZ$
- $X^\dagger X X = X$

In [38]:
def PauliAnticommute_X():
    thirdRootOfUnity = findAllRoots(3)
    w = thirdRootOfUnity[0]
    w_squared = thirdRootOfUnity[1]

    # XZ(X*) = w^2 Z
    LHS = X @ Z @ X.conj().T
    RHS = w_squared * Z
    equalUpToScalar(LHS, RHS)
    
    # XX(X*) = X
    LHS = X @ X @ X.conj().T
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (X*)ZX = w Z
    LHS = X.conj().T @ Z @ X
    RHS = w * Z
    equalUpToScalar(LHS, RHS)

    # (X*)XX = X
    LHS = X.conj().T @ X @ X
    RHS = X
    equalUpToScalar(LHS, RHS)

In [39]:
PauliAnticommute_X()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


# Pauli anticommutation: Z gate
- $ZXZ^\dagger = \omega X \iff ZX = \omega XZ$
- $ZZZ^\dagger = Z$
- $Z^\dagger X Z = \omega^2 X \iff XZ = \omega^2 ZX$
- $Z^\dagger Z Z = Z$

In [40]:
def PauliAnticommute_Z():
    thirdRootOfUnity = findAllRoots(3)
    w = thirdRootOfUnity[0]
    w_squared = thirdRootOfUnity[1]

    # ZX(Z*) = w X
    LHS = Z @ X @ Z.conj().T
    RHS = w * X
    equalUpToScalar(LHS, RHS)

    # ZZ(Z*) = Z
    LHS = Z @ Z @ Z.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (Z*)XZ = w^2 X
    LHS = Z.conj().T @ X @ Z
    RHS = w_squared * X
    equalUpToScalar(LHS, RHS)

    # (Z*)ZZ = Z
    LHS = Z.conj().T @ Z @ Z
    RHS = Z
    equalUpToScalar(LHS, RHS)

In [41]:
PauliAnticommute_Z()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


# The automorphism of a SWAP gate
- $SWAP(X\otimes I)SWAP^\dagger = (I \otimes X) \iff SWAP(X\otimes I) = (I \otimes X)SWAP$
- $SWAP(I\otimes X)SWAP^\dagger = (X \otimes I) \iff SWAP(I\otimes X) = (X \otimes I)SWAP$
- $SWAP(Z\otimes I)SWAP^\dagger = (I \otimes Z) \iff SWAP(Z\otimes I) = (I \otimes Z)SWAP$
- $SWAP(I\otimes Z)SWAP^\dagger = (Z \otimes I) \iff SWAP(I\otimes Z) = (Z \otimes I)SWAP$

In [42]:
def SWAP_Automorphism():
    I = np.identity(3)

    # SWAP(XxI)SWAP* = IxX
    LHS = SWAP @ np.kron(X, I) @ SWAP.conj().T
    RHS = np.kron(I, X)
    equalUpToScalar(LHS, RHS)
    
    # SWAP(IxX)SWAP* = XxI
    LHS = SWAP @ np.kron(I, X) @ SWAP.conj().T
    RHS = np.kron(X, I)
    equalUpToScalar(LHS, RHS)
    
    # SWAP(ZxI)SWAP* = IxZ
    LHS = SWAP @ np.kron(Z, I) @ SWAP.conj().T
    RHS = np.kron(I, Z)
    equalUpToScalar(LHS, RHS)
    
    # SWAP(IxZ)SWAP* = ZxI
    LHS = SWAP @ np.kron(I, Z) @ SWAP.conj().T
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

    # SWAP*(XxI)SWAP = IxX
    LHS = SWAP.conj().T @ np.kron(X, I) @ SWAP
    RHS = np.kron(I, X)
    equalUpToScalar(LHS, RHS)

    # SWAP*(IxX)SWAP = XxI
    LHS = SWAP.conj().T @ np.kron(I, X) @ SWAP
    RHS = np.kron(X, I)
    equalUpToScalar(LHS, RHS)

    # SWAP*(ZxI)SWAP = IxZ
    LHS = SWAP.conj().T @ np.kron(Z, I) @ SWAP
    RHS = np.kron(I, Z)
    equalUpToScalar(LHS, RHS)

    # SWAP*(IxZ)SWAP = ZxI
    LHS = SWAP.conj().T @ np.kron(I, Z) @ SWAP
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [43]:
SWAP_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


# The automorphism of a CZ gate
- $CZ(X\otimes I)CZ^\dagger = (X \otimes Z)$
- $CZ(I\otimes X)CZ^\dagger = (Z \otimes X)$
- $CZ(Z\otimes I)CZ^\dagger = (Z \otimes I)$
- $CZ(I\otimes Z)CZ^\dagger = (I \otimes Z)$
- $CZ^\dagger(X\otimes I)CZ = (X \otimes Z^2)$
- $CZ^\dagger(I\otimes X)CZ = (Z^2 \otimes X)$
- $CZ^\dagger(Z\otimes I)CZ = (Z \otimes I)$
- $CZ^\dagger(I\otimes Z)CZ = (I \otimes Z)$

In [44]:
def CZ_Automorphism():
    I = np.identity(3)
    # CZ(XxI)CZ* = XxZ
    LHS = CZ @ np.kron(X, I) @ CZ.conj().T
    RHS = np.kron(X, Z)
    equalUpToScalar(LHS, RHS)
    
    # CZ(IxX)CZ* = ZxX
    LHS = CZ @ np.kron(I, X) @ CZ.conj().T
    RHS = np.kron(Z, X)
    equalUpToScalar(LHS, RHS)
    
    # CZ(ZxI)CZ* = ZxI
    LHS = CZ @ np.kron(Z, I) @ CZ.conj().T
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)
    
    # CZ(IxZ)CZ* = IxZ
    LHS = CZ @ np.kron(I, Z) @ CZ.conj().T
    RHS = np.kron(I, Z)
    equalUpToScalar(LHS, RHS)

    # CZ*(XxI)CZ = (X)x(Z^2)
    LHS = CZ.conj().T @ np.kron(X, I) @ CZ
    RHS = np.kron(X, np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)
    
    # CZ*(IxX)CZ = (Z^2)x(X)
    LHS = CZ.conj().T @ np.kron(I, X) @ CZ
    RHS = np.kron(np.linalg.matrix_power(Z, 2), X)
    equalUpToScalar(LHS, RHS)

    # CZ*(ZxI)CZ = ZxI
    LHS = CZ.conj().T @ np.kron(Z, I) @ CZ
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)
    
    # CZ*(IxZ)CZ = IxZ
    LHS = CZ.conj().T @ np.kron(I, Z) @ CZ
    RHS = np.kron(I, Z) 
    equalUpToScalar(LHS, RHS)
    

In [45]:
CZ_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


# The automorphism of a Hadamard gate
- $HXH^\dagger = Z$
- $HZH^\dagger = X^2$
- $H^\dagger XH = Z^2$
- $H^\dagger ZH = X$

In [46]:
def Hadamard_Automorphism():
    # HXH* = Z
    LHS = H @ X @ H.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)
    
    # HZH* = X^2
    LHS = H @ Z @ H.conj().T
    RHS = X @ X
    equalUpToScalar(LHS, RHS)

    # H*XH = Z^2
    LHS = H.conj().T @ X @ H
    RHS = Z @ Z
    equalUpToScalar(LHS, RHS)
    
    # H*ZH = X
    LHS = H.conj().T @ Z @ H
    RHS = X
    equalUpToScalar(LHS, RHS)
    

In [47]:
Hadamard_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


# The automorphism of a Phase gate
- $SXS^\dagger = XZ^2$
- $SZS^\dagger = Z$
- $S^\dagger XS = XZ$
- $S^\dagger ZS = Z$

In [48]:
def Phase_Automorphism():
    # SXS* = XZ^2
    LHS = S @ X @ S.conj().T
    RHS = X @ np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)
    
    # SZS* = Z
    LHS = S @ Z @ S.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # S*XS = XZ
    LHS = S.conj().T @ X @ S
    RHS = X @ Z
    equalUpToScalar(LHS, RHS)
    
    # S*ZS = Z
    LHS = S.conj().T @ Z @ S
    RHS = Z
    equalUpToScalar(LHS, RHS)

In [49]:
Phase_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


# Scaled identity automorphism
For $t \in \mathbb{Z}_3$
- $(\omega^tI) X (\omega^tI)^\dagger = X$
- $(\omega^tI) Z (\omega^tI)^\dagger = Z$
- $(\omega^tI)^\dagger X (\omega^tI) = X$
- $(\omega^tI)^\dagger Z (\omega^tI) = Z$

In [50]:
def scaledIdentity_Automorphism():
    w = findAllRoots(3)[1]
    wI = makeScalarId(3, w)

    # (wI)X(wI)* = X
    LHS = wI @ X @ wI.conj().T
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (wI)Z(wI)* = Z
    LHS = wI @ Z @ wI.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (wI)*X(wI) = X
    LHS = wI.conj().T @ X @ wI
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (wI)*Z(wI) = Z
    LHS = wI.conj().T @ Z @ wI
    RHS = Z
    equalUpToScalar(LHS, RHS)

In [51]:
scaledIdentity_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


# A boxes
- $A_{01} = I$
- $A_{02} = H^2$
- $A_{10} = H$
- $A_{11} = HS$
- $A_{12} = HS^2$
- $A_{20} = H^3 = HH^2$
- $A_{21} = HS^2H^2$
- $A_{22} = HSH^2$ 

In [52]:
I = np.identity(3)
A01 = I
A02 = np.linalg.matrix_power(H, 2)
A10 = H
A11 = H @ S
A12 = H @ np.linalg.matrix_power(S, 2)
A20 = H @ np.linalg.matrix_power(H, 2)
A21 = H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2)
A22 = H @ S @ np.linalg.matrix_power(H, 2)

# Verify the automorphism of each A box

In [53]:
def A01_Automorphism():
    A = A01
    # (A)X(A)* = X
    LHS = A @ X @ A.conj().T
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (A)Z(A)* = Z
    LHS = A @ Z @ A.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (A)*X(A) = X
    LHS = A.conj().T @ X @ A
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (A)*Z(A) = Z
    LHS = A.conj().T @ Z @ A
    RHS = Z
    equalUpToScalar(LHS, RHS)

# C boxes

In [54]:
I = np.identity(3)
C0 = I
C1 = H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2) @ S @ H
C2 = H @ S @ np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2) @ H

# E boxes

In [55]:
I = np.identity(3)
E0 = I
E1 = S
E2 = S @ S

# F boxes

In [56]:
I = np.identity(3)
F0 = I
F1 = np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)
F2 = S @ np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2)

In [57]:
I = np.identity(3)
w = findAllRoots(3)[1]
wI = makeScalarId(3, w)
LHS = np.linalg.matrix_power(S @ H, 3)
RHS = wI
equalUpToScalar(LHS, RHS)

Not equal


False

# Verify the automorphism of each F box

In [58]:
def F1_Automorphism():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    F = F1
    # (F)X(F)* = w^2X
    LHS = F @ X @ F.conj().T
    RHS = w2I @ X
    equalUpToScalar(LHS, RHS)

    # (F)Z(F)* = Z
    LHS = F @ Z @ F.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (F)*X(F) = wX
    LHS = F.conj().T @ X @ F
    RHS = wI @ X
    equalUpToScalar(LHS, RHS)

    # (F)*Z(F) = Z
    LHS = F.conj().T @ Z @ F
    RHS = Z
    equalUpToScalar(LHS, RHS)

In [59]:
def F2_Automorphism():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    F = F2
    # (F)X(F)* = wX
    LHS = F @ X @ F.conj().T
    RHS = wI @ X
    equalUpToScalar(LHS, RHS)

    # (F)Z(F)* = Z
    LHS = F @ Z @ F.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (F)*X(F) = w^2X
    LHS = F.conj().T @ X @ F
    RHS = w2I @ X
    equalUpToScalar(LHS, RHS)

    # (F)*Z(F) = Z
    LHS = F.conj().T @ Z @ F
    RHS = Z
    equalUpToScalar(LHS, RHS)

In [60]:
def F0_Automorphism():
    F = F0
    # (F)X(F)* = X
    LHS = F @ X @ F.conj().T
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (F)Z(F)* = Z
    LHS = F @ Z @ F.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (F)*X(F) = X
    LHS = F.conj().T @ X @ F
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (F)*Z(F) = Z
    LHS = F.conj().T @ Z @ F
    RHS = Z
    equalUpToScalar(LHS, RHS)

In [61]:
F0_Automorphism()
F1_Automorphism()
F2_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


# Verify the automorphism of each E box

In [62]:
def E0_Automorphism():
    E = E0
    # (E)X(E)* = X
    LHS = E @ X @ E.conj().T
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (E)Z(E)* = Z
    LHS = E @ Z @ E.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (E)*X(E) = X
    LHS = E.conj().T @ X @ E
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (E)*Z(E) = Z
    LHS = E.conj().T @ Z @ E
    RHS = Z
    equalUpToScalar(LHS, RHS)

In [63]:
E0_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [64]:
def E1_Automorphism():
    E = E1
    # (E)X(E)* = XZ^2
    LHS = E @ X @ E.conj().T
    RHS = X @ np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)

    # (E)Z(E)* = Z
    LHS = E @ Z @ E.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (E)*X(E) = XZ
    LHS = E.conj().T @ X @ E
    RHS = X @ Z
    equalUpToScalar(LHS, RHS)

    # (E)*Z(E) = Z
    LHS = E.conj().T @ Z @ E
    RHS = Z
    equalUpToScalar(LHS, RHS)

In [65]:
E1_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [66]:
def E2_Automorphism():
    E = E2
    # (E)X(E)* = XZ
    LHS = E @ X @ E.conj().T
    RHS = X @ Z
    equalUpToScalar(LHS, RHS)

    # (E)Z(E)* = Z
    LHS = E @ Z @ E.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (E)*X(E) = XZ^2
    LHS = E.conj().T @ X @ E
    RHS = X @ Z @ Z
    equalUpToScalar(LHS, RHS)

    # (E)*Z(E) = Z
    LHS = E.conj().T @ Z @ E
    RHS = Z
    equalUpToScalar(LHS, RHS)

In [67]:
E2_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


# Verify the automorphism of each C box

In [68]:
def C0_Automorphism():
    C = C0
    # (C)X(C)* = X
    LHS = C @ X @ C.conj().T
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (C)Z(C)* = Z
    LHS = C @ Z @ C.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (C)*X(C) = X
    LHS = C.conj().T @ X @ C
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (C)*Z(C) = Z
    LHS = C.conj().T @ Z @ C
    RHS = Z
    equalUpToScalar(LHS, RHS)

In [69]:
C0_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [70]:
def C1_Automorphism():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    C = C1
    # (C)X(C)* = X
    LHS = C @ X @ C.conj().T
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (C)Z(C)* = w^2Z
    LHS = C @ Z @ C.conj().T
    RHS = Z @ w2I
    equalUpToScalar(LHS, RHS)

    # (C)*X(C) = X
    LHS = C.conj().T @ X @ C
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (C)*Z(C) = wZ
    LHS = C.conj().T @ Z @ C
    RHS = Z @ wI
    equalUpToScalar(LHS, RHS)

In [71]:
C1_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [72]:
def C2_Automorphism():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    C = C2
    # (C)X(C)* = X
    LHS = C @ X @ C.conj().T
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (C)Z(C)* = wZ
    LHS = C @ Z @ C.conj().T
    RHS = Z @ wI
    equalUpToScalar(LHS, RHS)

    # (C)*X(C) = X
    LHS = C.conj().T @ X @ C
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (C)*Z(C) = w^2Z
    LHS = C.conj().T @ Z @ C
    RHS = Z @ w2I
    equalUpToScalar(LHS, RHS)

In [73]:
C2_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [74]:
def A02_Automorphism():
    A = A02
    # (A)X(A)* = X^2
    LHS = A @ X @ A.conj().T
    RHS = np.linalg.matrix_power(X, 2)
    equalUpToScalar(LHS, RHS)

    # (A)Z(A)* = Z^2
    LHS = A @ Z @ A.conj().T
    RHS = np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)

    # (A)*X(A) = X^2
    LHS = A.conj().T @ X @ A
    RHS = np.linalg.matrix_power(X, 2)
    equalUpToScalar(LHS, RHS)

    # (A)*Z(A) = Z^2
    LHS = A.conj().T @ Z @ A
    RHS = np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)

In [75]:
A02_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [76]:
def A10_Automorphism():
    A = A10
    # (A)X(A)* = Z
    LHS = A @ X @ A.conj().T
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (A)Z(A)* = X^2
    LHS = A @ Z @ A.conj().T
    RHS = np.linalg.matrix_power(X, 2)
    equalUpToScalar(LHS, RHS)

    # (A)*X(A) = Z^2
    LHS = A.conj().T @ X @ A
    RHS = np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)

    # (A)*Z(A) = X
    LHS = A.conj().T @ Z @ A
    RHS = X
    equalUpToScalar(LHS, RHS)

In [77]:
A10_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [78]:
def A11_Automorphism():
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    A = A11
    # (A)X(A)* = wXZ
    LHS = A @ X @ A.conj().T
    RHS = wI @ X @ Z
    equalUpToScalar(LHS, RHS)

    # (A)Z(A)* = X^2
    LHS = A @ Z @ A.conj().T
    RHS = np.linalg.matrix_power(X, 2)
    equalUpToScalar(LHS, RHS)

    # (A)*X(A) = Z^2
    LHS = A.conj().T @ X @ A
    RHS = np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)

    # (A)*Z(A) = XZ
    LHS = A.conj().T @ Z @ A
    RHS = X @ Z
    equalUpToScalar(LHS, RHS)

In [79]:
A11_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [80]:
def A12_Automorphism():
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    A = A12
    
    # (A)X(A)* = w^2X^2Z
    LHS = A @ X @ A.conj().T
    RHS = w2I @ np.linalg.matrix_power(X, 2) @ Z
    equalUpToScalar(LHS, RHS)

    # (A)Z(A)* = X^2
    LHS = A @ Z @ A.conj().T
    RHS = np.linalg.matrix_power(X, 2)
    equalUpToScalar(LHS, RHS)

    # (A)*X(A) = Z^2
    LHS = A.conj().T @ X @ A
    RHS = np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)

    # (A)*Z(A) = XZ^2
    LHS = A.conj().T @ Z @ A
    RHS = X @ np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)

In [81]:
A12_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [82]:
def A20_Automorphism():

    A = A20
    
    # (A)X(A)* = Z^2
    LHS = A @ X @ A.conj().T
    RHS = np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)

    # (A)Z(A)* = X
    LHS = A @ Z @ A.conj().T
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (A)*X(A) = Z
    LHS = A.conj().T @ X @ A
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (A)*Z(A) = X^2
    LHS = A.conj().T @ Z @ A
    RHS = np.linalg.matrix_power(X, 2)
    equalUpToScalar(LHS, RHS)

In [83]:
A20_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [84]:
def A21_Automorphism():

    A = A21
    
    # (A)X(A)* = XZ^2
    LHS = A @ X @ A.conj().T
    RHS = X @ np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)

    # (A)Z(A)* = X
    LHS = A @ Z @ A.conj().T
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (A)*X(A) = Z
    LHS = A.conj().T @ X @ A
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (A)*Z(A) = X^2Z
    LHS = A.conj().T @ Z @ A
    RHS = np.linalg.matrix_power(X, 2) @ Z
    equalUpToScalar(LHS, RHS)

In [85]:
A21_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [86]:
def A22_Automorphism():

    A = A22
    
    # (A)X(A)* = X^2Z^2
    LHS = A @ X @ A.conj().T
    RHS = np.linalg.matrix_power(X, 2) @ np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)

    # (A)Z(A)* = X
    LHS = A @ Z @ A.conj().T
    RHS = X
    equalUpToScalar(LHS, RHS)

    # (A)*X(A) = Z
    LHS = A.conj().T @ X @ A
    RHS = Z
    equalUpToScalar(LHS, RHS)

    # (A)*Z(A) = X^2Z^2
    LHS = A.conj().T @ Z @ A
    RHS = np.linalg.matrix_power(X, 2) @ np.linalg.matrix_power(Z, 2)
    equalUpToScalar(LHS, RHS)

In [87]:
A22_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


# Push an H gate through an A box
- $A_{01}H = A_{10}$
- $A_{02}H = A_{20}$
- $A_{10}H = A_{02}$
- $A_{11}H = X^2 S^2 A_{12}$
- $A_{12}H = S X A_{22}$
- $A_{20}H = A_{01}$
- $A_{21}H = S X A_{11}$
- $A_{22}H = X^2 S^2 A_{21}$

In [88]:
def H_A_boxes():
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    LHS = A01 @ H
    RHS = A10
    equalUpToScalar(LHS, RHS)

    LHS = A02 @ H
    RHS = A20
    equalUpToScalar(LHS, RHS)

    LHS = A10 @ H
    RHS = A02
    equalUpToScalar(LHS, RHS)

    LHS = A11 @ H
    RHS = np.linalg.matrix_power(X, 2) @ np.linalg.matrix_power(S, 2) @ A12 @ w2I * (-1)
    equalUpToScalar(LHS, RHS)

    LHS = A12 @ H
    RHS = S @ X @ A22 @ wI * (-1)
    equalUpToScalar(LHS, RHS)

    LHS = A20 @ H
    RHS = A01
    equalUpToScalar(LHS, RHS)

    LHS = A21 @ H
    RHS = S @ X @ A11 @ wI * (-1)
    equalUpToScalar(LHS, RHS)

    LHS = A22 @ H
    RHS = np.linalg.matrix_power(X, 2) @ np.linalg.matrix_power(S, 2) @ A21 @ w2I * (-1)
    equalUpToScalar(LHS, RHS)

In [89]:
H_A_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [90]:
def S_A_boxes():
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    LHS = A01 @ S
    RHS = S @ A01
    equalUpToScalar(LHS, RHS)

    LHS = A02 @ S
    RHS = np.linalg.matrix_power(Z, 2) @ S @ A02
    equalUpToScalar(LHS, RHS)

    LHS = A10 @ S
    RHS = A11
    equalUpToScalar(LHS, RHS)

    LHS = A11 @ S
    RHS = A12
    equalUpToScalar(LHS, RHS)

    LHS = A12 @ S
    RHS = A10
    equalUpToScalar(LHS, RHS)

    LHS = A20 @ S
    RHS = X @ A22
    equalUpToScalar(LHS, RHS)

    LHS = A21 @ S
    RHS = X @ A20
    equalUpToScalar(LHS, RHS)

    LHS = A22 @ S
    RHS = X @ A21
    equalUpToScalar(LHS, RHS)

In [91]:
S_A_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


# D boxes
- $D_{00} = SWAP$
- $D_{01} = CZ^2\circ SWAP$
- $D_{02} = CZ\circ SWAP$
- $D_{10} = D_{01}\circ (I \otimes A_{10})$
- $D_{11} = D_{01}\circ (I \otimes A_{11})$
- $D_{12} = D_{01}\circ (I \otimes A_{12})$
- $D_{20} = D_{01}\circ (I \otimes A_{20})$
- $D_{21} = D_{01}\circ (I \otimes A_{21})$
- $D_{22} = D_{01}\circ (I \otimes A_{22})$ 

In [92]:
I = np.identity(3)
D00 = SWAP
D01 = np.linalg.matrix_power(CZ, 2) @ SWAP
D02 = CZ @ SWAP
D10 = D01 @ np.kron(I, A10)
D11 = D01 @ np.kron(I, A11)
D12 = D01 @ np.kron(I, A12)
D20 = D01 @ np.kron(I, A20)
D21 = D01 @ np.kron(I, A21)
D22 = D01 @ np.kron(I, A22)

# Verify the automorphism of each D box

In [93]:
def D00_Automorphism():
    D = D00
    I = np.identity(3)
    # (D)(X x I)(D)* = I x X
    LHS = D @ np.kron(X, I) @ D.conj().T
    RHS = np.kron(I, X)
    equalUpToScalar(LHS, RHS)

    # (D)(I x X)(D)* = X x I
    LHS = D @ np.kron(I, X) @ D.conj().T
    RHS = np.kron(X, I)
    equalUpToScalar(LHS, RHS)

    # (D)(Z x I)(D)* = I x Z
    LHS = D @ np.kron(Z, I) @ D.conj().T
    RHS = np.kron(I, Z)
    equalUpToScalar(LHS, RHS)

    # (D)(I x Z)(D)* = Z x I
    LHS = D @ np.kron(I, Z) @ D.conj().T
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

    # (D)*(X x I)(D) = I x X
    LHS = D.conj().T @ np.kron(X, I) @ D
    RHS = np.kron(I, X)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x X)(D) = X x I
    LHS = D.conj().T @ np.kron(I, X) @ D
    RHS = np.kron(X, I)
    equalUpToScalar(LHS, RHS)

    # (D)*(Z x I)(D) = I x Z
    LHS = D.conj().T @ np.kron(Z, I) @ D
    RHS = np.kron(I, Z)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x Z)(D) = Z x I
    LHS = D.conj().T @ np.kron(I, Z) @ D
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [94]:
D00_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [95]:
def D01_Automorphism():
    D = D01
    I = np.identity(3)
    # (D)(X x I)(D)* = Z^2 x X
    LHS = D @ np.kron(X, I) @ D.conj().T
    RHS = np.kron(np.linalg.matrix_power(Z, 2), X)
    equalUpToScalar(LHS, RHS)

    # (D)(I x X)(D)* = X x Z^2
    LHS = D @ np.kron(I, X) @ D.conj().T
    RHS = np.kron(X, np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (D)(Z x I)(D)* = I x Z
    LHS = D @ np.kron(Z, I) @ D.conj().T
    RHS = np.kron(I, Z)
    equalUpToScalar(LHS, RHS)

    # (D)(I x Z)(D)* = Z x I
    LHS = D @ np.kron(I, Z) @ D.conj().T
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

    # (D)*(X x I)(D) = Z x X
    LHS = D.conj().T @ np.kron(X, I) @ D
    RHS = np.kron(Z, X)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x X)(D) = X x Z
    LHS = D.conj().T @ np.kron(I, X) @ D
    RHS = np.kron(X, Z)
    equalUpToScalar(LHS, RHS)

    # (D)*(Z x I)(D) = I x Z
    LHS = D.conj().T @ np.kron(Z, I) @ D
    RHS = np.kron(I, Z)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x Z)(D) = Z x I
    LHS = D.conj().T @ np.kron(I, Z) @ D
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [96]:
D01_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [97]:
def D02_Automorphism():
    D = D02
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (D)*(I x X)(D) = X x Z^2
    LHS = D.conj().T @ np.kron(I, X) @ D
    RHS = np.kron(X, np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ)(D) = XZ x Z^2
    LHS = D.conj().T @ np.kron(I, X @ Z) @ D
    RHS = np.kron(X @ Z, np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ^2)(D) = XZ^2 x Z^2
    LHS = D.conj().T @ np.kron(I, X @ np.linalg.matrix_power(Z, 2)) @ D
    RHS = np.kron(X @ np.linalg.matrix_power(Z, 2), np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x Z)(D) = Z x I
    LHS = D.conj().T @ np.kron(I, Z) @ D
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [98]:
D02_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [99]:
def D10_Automorphism():
    D = D10
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (D)*(I x X)(D) = X x X
    LHS = D.conj().T @ np.kron(I, X) @ D
    RHS = np.kron(X, X)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ)(D) = XZ x X
    LHS = D.conj().T @ np.kron(I, X @ Z) @ D
    RHS = np.kron(X @ Z, X)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ^2)(D) = XZ^2 x X
    LHS = D.conj().T @ np.kron(I, X @ np.linalg.matrix_power(Z, 2)) @ D
    RHS = np.kron(X @ np.linalg.matrix_power(Z, 2), X)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x Z)(D) = Z x I
    LHS = D.conj().T @ np.kron(I, Z) @ D
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [100]:
D10_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [101]:
def D11_Automorphism():
    D = D11
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (D)*(I x X)(D) = X x XZ
    LHS = D.conj().T @ np.kron(I, X) @ D
    RHS = np.kron(X, X @ Z)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ)(D) = XZ x XZ
    LHS = D.conj().T @ np.kron(I, X @ Z) @ D
    RHS = np.kron(X @ Z, X @ Z)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ^2)(D) = XZ^2 x XZ
    LHS = D.conj().T @ np.kron(I, X @ np.linalg.matrix_power(Z, 2)) @ D
    RHS = np.kron(X @ np.linalg.matrix_power(Z, 2), X @ Z)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x Z)(D) = Z x I
    LHS = D.conj().T @ np.kron(I, Z) @ D
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [102]:
D11_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [103]:
def D12_Automorphism():
    D = D12
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (D)*(I x X)(D) = X x XZ^2
    LHS = D.conj().T @ np.kron(I, X) @ D
    RHS = np.kron(X, X @ np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ)(D) = XZ x XZ^2
    LHS = D.conj().T @ np.kron(I, X @ Z) @ D
    RHS = np.kron(X @ Z, X @ np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ^2)(D) = XZ^2 x XZ^2
    LHS = D.conj().T @ np.kron(I, X @ np.linalg.matrix_power(Z, 2)) @ D
    RHS = np.kron(X @ np.linalg.matrix_power(Z, 2), X @ np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x Z)(D) = Z x I
    LHS = D.conj().T @ np.kron(I, Z) @ D
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [104]:
D12_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [105]:
def D20_Automorphism():
    D = D20
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (D)*(I x X)(D) = X x X^2
    LHS = D.conj().T @ np.kron(I, X) @ D
    RHS = np.kron(X, np.linalg.matrix_power(X, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ)(D) = XZ x X^2
    LHS = D.conj().T @ np.kron(I, X @ Z) @ D
    RHS = np.kron(X @ Z, np.linalg.matrix_power(X, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ^2)(D) = XZ^2 x X^2
    LHS = D.conj().T @ np.kron(I, X @ np.linalg.matrix_power(Z, 2)) @ D
    RHS = np.kron(X @ np.linalg.matrix_power(Z, 2), np.linalg.matrix_power(X, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x Z)(D) = Z x I
    LHS = D.conj().T @ np.kron(I, Z) @ D
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [106]:
D20_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [107]:
def D21_Automorphism():
    D = D21
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (D)*(I x X)(D) = X x X^2Z
    LHS = D.conj().T @ np.kron(I, X) @ D
    RHS = np.kron(X, np.linalg.matrix_power(X, 2) @ Z)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ)(D) = XZ x X^2Z
    LHS = D.conj().T @ np.kron(I, X @ Z) @ D
    RHS = np.kron(X @ Z, np.linalg.matrix_power(X, 2) @ Z)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ^2)(D) = XZ^2 x X^2Z
    LHS = D.conj().T @ np.kron(I, X @ np.linalg.matrix_power(Z, 2)) @ D
    RHS = np.kron(X @ np.linalg.matrix_power(Z, 2), np.linalg.matrix_power(X, 2) @ Z)
    equalUpToScalar(LHS, RHS)

    # (D)*(I x Z)(D) = Z x I
    LHS = D.conj().T @ np.kron(I, Z) @ D
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [108]:
D21_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [109]:
def D22_Automorphism():
    D = D22
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (D)*(I x X)(D) = X x X^2Z^2
    LHS = D.conj().T @ np.kron(I, X) @ D
    RHS = np.kron(X, np.linalg.matrix_power(X, 2) @ np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ)(D) = XZ x X^2Z^2
    LHS = D.conj().T @ np.kron(I, X @ Z) @ D
    RHS = np.kron(X @ Z, np.linalg.matrix_power(X, 2) @ np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x XZ^2)(D) = XZ^2 x X^2Z^2
    LHS = D.conj().T @ np.kron(I, X @ np.linalg.matrix_power(Z, 2)) @ D
    RHS = np.kron(X @ np.linalg.matrix_power(Z, 2), np.linalg.matrix_power(X, 2) @ np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (D)*(I x Z)(D) = Z x I
    LHS = D.conj().T @ np.kron(I, Z) @ D
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [110]:
D22_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal


# B boxes
- $B_{00} = CZ\circ SWAP$
- $B_{01} = (H^3 \otimes I)\circ CZ \circ(H\otimes I)\circ CZ^2\circ SWAP$
- $B_{02} = B_{01}\circ CZ \circ (A_{02}\otimes I)$
- $B_{10} = B_{01}\circ CZ^2 \circ (A_{10}\otimes I)$
- $B_{11} = B_{01}\circ CZ^2 \circ (A_{11}\otimes I)$
- $B_{12} = B_{01}\circ CZ^2 \circ (A_{12}\otimes I)$
- $B_{20} = B_{01}\circ CZ^2 \circ (A_{20}\otimes I)$
- $B_{21} = B_{01}\circ CZ^2 \circ (A_{21}\otimes I)$
- $B_{22} = B_{01}\circ CZ^2 \circ (A_{22}\otimes I)$ 

In [111]:
I = np.identity(3)
B00 = CZ @ SWAP
B01 = np.kron(np.linalg.matrix_power(H, 3), I) @ CZ @ np.kron(H, I) @ np.linalg.matrix_power(CZ, 2) @ SWAP
B02 = B01 @ CZ @ np.kron(A02, I)
B10 = B01 @ np.linalg.matrix_power(CZ, 2) @ np.kron(A10, I)
B11 = B01 @ np.linalg.matrix_power(CZ, 2) @ np.kron(A11, I)
B12 = B01 @ np.linalg.matrix_power(CZ, 2) @ np.kron(A12, I)
B20 = B01 @ np.linalg.matrix_power(CZ, 2) @ np.kron(A20, I)
B21 = B01 @ np.linalg.matrix_power(CZ, 2) @ np.kron(A21, I)
B22 = B01 @ np.linalg.matrix_power(CZ, 2) @ np.kron(A22, I)

# Verify the automorphism of each B box

In [112]:
def B00_Automorphism():
    B = B00
    
    # (B)(X x I)(B)* = Z x X
    LHS = B @ np.kron(X, I) @ B.conj().T
    RHS = np.kron(Z, X)
    equalUpToScalar(LHS, RHS)

    # (B)(I x X)(B)* = X x Z
    LHS = B @ np.kron(I, X) @ B.conj().T
    RHS = np.kron(X, Z)
    equalUpToScalar(LHS, RHS)

    # (B)(Z x I)(B)* = I x Z
    LHS = B @ np.kron(Z, I) @ B.conj().T
    RHS = np.kron(I, Z)
    equalUpToScalar(LHS, RHS)

    # (B)(I x Z)(B)* = Z x I
    LHS = B @ np.kron(I, Z) @ B.conj().T
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

    # (B)*(X x I)(B) = Z^2 x X
    LHS = B.conj().T @ np.kron(X, I) @ B
    RHS = np.kron(np.linalg.matrix_power(Z, 2), X)
    equalUpToScalar(LHS, RHS)

    # (B)*(I x X)(B) = X x Z^2
    LHS = B.conj().T @ np.kron(I, X) @ B
    RHS = np.kron(X, np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (B)*(Z x I)(B) = I x Z
    LHS = B.conj().T @ np.kron(Z, I) @ B
    RHS = np.kron(I, Z)
    equalUpToScalar(LHS, RHS)

    # (B)*(I x Z)(B) = Z x I
    LHS = B.conj().T @ np.kron(I, Z) @ B
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [113]:
B00_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [114]:
def B01_Automorphism():
    B = B01
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (B)(X x I)(B)* = XZ^2 x XZ
    LHS = B @ np.kron(X, I) @ B.conj().T
    RHS = np.kron(X @ np.linalg.matrix_power(Z, 2), X @ Z)
    equalUpToScalar(LHS, RHS)

    # (B)(I x X)(B)* = X x Z^2
    LHS = B @ np.kron(I, X) @ B.conj().T
    RHS = np.kron(X, np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (B)(Z x I)(B)* = I x Z
    LHS = B @ np.kron(Z, I) @ B.conj().T
    RHS = np.kron(I, Z)
    equalUpToScalar(LHS, RHS)

    # (B)(I x Z)(B)* = Z x Z^2
    LHS = B @ np.kron(I, Z) @ B.conj().T
    RHS = np.kron(Z, np.linalg.matrix_power(Z, 2))
    equalUpToScalar(LHS, RHS)

    # (B)*(X x I)(B) = Z x X
    LHS = B.conj().T @ np.kron(X, I) @ B
    RHS = np.kron(Z, X)
    equalUpToScalar(LHS, RHS)

    # (B)*(I x X)(B) = XZ^2 x w^2X^2Z
    LHS = B.conj().T @ np.kron(I, X) @ B
    RHS = np.kron(X @ np.linalg.matrix_power(Z, 2), w2I @ np.linalg.matrix_power(X, 2) @ Z)
    equalUpToScalar(LHS, RHS)

    # (B)*(Z x I)(B) = Z x Z
    LHS = B.conj().T @ np.kron(Z, I) @ B
    RHS = np.kron(Z, Z)
    equalUpToScalar(LHS, RHS)

    # (B)*(I x Z)(B) = Z x I
    LHS = B.conj().T @ np.kron(I, Z) @ B
    RHS = np.kron(Z, I)
    equalUpToScalar(LHS, RHS)

In [115]:
B01_Automorphism()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [116]:
def B02_Automorphism():
    B = B02
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (B)*(Z x I)(B) = Z^2 x Z
    LHS = B.conj().T @ np.kron(Z, I) @ B
    RHS = np.kron(np.linalg.matrix_power(Z, 2), Z)
    equalUpToScalar(LHS, RHS)

In [117]:
B02_Automorphism()

Exactly equal


In [118]:
def B10_Automorphism():
    B = B10
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (B)*(Z x I)(B) = X x Z
    LHS = B.conj().T @ np.kron(Z, I) @ B
    RHS = np.kron(X, Z)
    equalUpToScalar(LHS, RHS)

In [119]:
B10_Automorphism()

Exactly equal


In [120]:
def B11_Automorphism():
    B = B11
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (B)*(Z x I)(B) = XZ x Z
    LHS = B.conj().T @ np.kron(Z, I) @ B
    RHS = np.kron(X @ Z, Z)
    equalUpToScalar(LHS, RHS)

In [121]:
B11_Automorphism()

Exactly equal


In [122]:
def B12_Automorphism():
    B = B12
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (B)*(Z x I)(B) = XZ^2 x Z
    LHS = B.conj().T @ np.kron(Z, I) @ B
    RHS = np.kron(X @ np.linalg.matrix_power(Z, 2), Z)
    equalUpToScalar(LHS, RHS)

In [123]:
B12_Automorphism()

Exactly equal


In [124]:
def B20_Automorphism():
    B = B20
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (B)*(Z x I)(B) = X^2 x Z
    LHS = B.conj().T @ np.kron(Z, I) @ B
    RHS = np.kron(np.linalg.matrix_power(X, 2), Z)
    equalUpToScalar(LHS, RHS)

In [125]:
B20_Automorphism()

Exactly equal


In [126]:
def B21_Automorphism():
    B = B21
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (B)*(Z x I)(B) = X^2Z x Z
    LHS = B.conj().T @ np.kron(Z, I) @ B
    RHS = np.kron(np.linalg.matrix_power(X, 2) @ Z, Z)
    equalUpToScalar(LHS, RHS)

In [127]:
B21_Automorphism()

Exactly equal


In [128]:
def B22_Automorphism():
    B = B22
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)
    
    # (B)*(Z x I)(B) = X^2Z^2 x Z
    LHS = B.conj().T @ np.kron(Z, I) @ B
    RHS = np.kron(np.linalg.matrix_power(X, 2) @ np.linalg.matrix_power(Z, 2), Z)
    equalUpToScalar(LHS, RHS)

In [129]:
B22_Automorphism()

Exactly equal


In [130]:
def CZ_A_boxes():
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    residual1 = np.kron(I, Z @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2) @ np.kron(S, H @ np.linalg.matrix_power(S, 2))
    residual2 = np.kron(I, np.linalg.matrix_power(S, 2) @ H) @ np.linalg.matrix_power(CZ, 2) @ np.kron(S, H @ np.linalg.matrix_power(S, 2))
    
    LHS = np.kron(A01, I) @ CZ 
    RHS = CZ @ np.kron(A01, I)
    equalUpToScalar(LHS, RHS)

    LHS = np.kron(A02, I) @ CZ 
    RHS = np.linalg.matrix_power(CZ, 2) @ np.kron(A02, I)
    equalUpToScalar(LHS, RHS)

    LHS = np.kron(A10, I) @ CZ 
    RHS = residual1 @ B10 @ np.kron(I, A02) @ wI
    equalUpToScalar(LHS, RHS)

    LHS = np.kron(A11, I) @ CZ 
    RHS = residual1 @ B11 @ np.kron(I, A02) @ wI
    equalUpToScalar(LHS, RHS)

    LHS = np.kron(A12, I) @ CZ 
    RHS = residual1 @ B12 @ np.kron(I, A02) @ wI
    equalUpToScalar(LHS, RHS)

    LHS = np.kron(A20, I) @ CZ 
    RHS = residual2 @ B20 @ np.kron(I, A01) @ wI
    equalUpToScalar(LHS, RHS)

    LHS = np.kron(A21, I) @ CZ 
    RHS = residual2 @ B21 @ np.kron(I, A01) @ wI
    equalUpToScalar(LHS, RHS)

    LHS = np.kron(A22, I) @ CZ 
    RHS = residual2 @ B22 @ np.kron(I, A01) @ wI
    equalUpToScalar(LHS, RHS)

In [131]:
CZ_A_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [132]:
def CZ_A01_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    LHS = B00 @ np.kron(I, A01) @ CZ
    RHS = CZ @ B00 @ np.kron(I, A01)
    equalUpToScalar(LHS, RHS)
    
    residual1 = np.kron(I, S @ np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2)) @ CZ
    LHS = B01 @ np.kron(I, A01) @ CZ
    RHS = residual1 @ B01 @ np.kron(I, A01) @ wI
    equalUpToScalar(LHS, RHS)

    residual2 = np.linalg.matrix_power(CZ, 2) @ np.kron(I, Z @ S)
    LHS = B02 @ np.kron(I, A01) @ CZ
    RHS = residual2 @ B02 @ np.kron(I, A01) @ w2I
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, S @ H) @ np.linalg.matrix_power(CZ, 2) @ np.kron(S @ S, H @ S)
    LHS = B10 @ np.kron(I, A01) @ CZ
    RHS = residual3 @ np.kron(A10, I) @ w2I
    equalUpToScalar(LHS, RHS)

    LHS = B11 @ np.kron(I, A01) @ CZ
    RHS = residual3 @ np.kron(A11, I) @ w2I
    equalUpToScalar(LHS, RHS)

    LHS = B12 @ np.kron(I, A01) @ CZ
    RHS = residual3 @ np.kron(A12, I) @ w2I
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(I, Z @ S @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2) @ np.kron(I, H @ S @ np.linalg.matrix_power(H, 2) @ S)
    LHS = B20 @ np.kron(I, A01) @ CZ
    RHS = residual4 @ B20 @ np.kron(I, A02)
    equalUpToScalar(LHS, RHS)

    LHS = B21 @ np.kron(I, A01) @ CZ
    RHS = residual4 @ B21 @ np.kron(I, A02)
    equalUpToScalar(LHS, RHS)

    LHS = B22 @ np.kron(I, A01) @ CZ
    RHS = residual4 @ B22 @ np.kron(I, A02)
    equalUpToScalar(LHS, RHS)

In [133]:
CZ_A01_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [134]:
def CZ_A02_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    # Relation 1
    LHS = B00 @ np.kron(I, A02) @ CZ
    RHS = np.linalg.matrix_power(CZ, 2) @ B00 @ np.kron(I, A02)
    equalUpToScalar(LHS, RHS)
    
    residual1 = np.kron(I, np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2)) @ np.linalg.matrix_power(CZ, 2)

    # Relation 2
    LHS = B01 @ np.kron(I, A02) @ CZ
    RHS = residual1 @ B01 @ np.kron(I, A02) @ w2I
    equalUpToScalar(LHS, RHS)

    residual2 = CZ @ np.kron(I, np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(S, 2))
    
    # Relation 3
    LHS = B02 @ np.kron(I, A02) @ CZ
    RHS = residual2 @ B02 @ np.kron(I, A02) @ wI
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, Z @ S @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2) @ np.kron(I, H @ S @ np.linalg.matrix_power(H, 2) @ S)
    
    # Relation 4
    LHS = B10 @ np.kron(I, A02) @ CZ
    RHS = residual3 @ B10 @ np.kron(I, A01)
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = B11 @ np.kron(I, A02) @ CZ
    RHS = residual3 @ B11 @ np.kron(I, A01)
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = B12 @ np.kron(I, A02) @ CZ
    RHS = residual3 @ B12 @ np.kron(I, A01)
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(I, S @ H) @ np.linalg.matrix_power(CZ, 2) @ np.kron(np.linalg.matrix_power(S, 2), H @ S @ np.linalg.matrix_power(H, 2))
    # Relation 7
    LHS = B20 @ np.kron(I, A02) @ CZ
    RHS = residual4 @ np.kron(A20, I) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(I, A02) @ CZ
    RHS = residual4 @ np.kron(A21, I) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = B22 @ np.kron(I, A02) @ CZ
    RHS = residual4 @ np.kron(A22, I) @ w2I
    equalUpToScalar(LHS, RHS)

In [135]:
CZ_A02_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [136]:
def CZ_A10_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    residual1 = np.kron(I, np.linalg.matrix_power(H, 2)) @ np.linalg.matrix_power(CZ, 2)
    # Relation 1
    LHS = B00 @ np.kron(I, A10) @ CZ
    RHS = residual1 @ B02 @ np.kron(I, A10)
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2) 
                        @ np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2)) @ CZ
    # Relation 2
    LHS = B01 @ np.kron(I, A10) @ CZ
    RHS = residual2 @ B00 @ np.kron(I, A10) @ w2I
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, S @ np.linalg.matrix_power(H, 2) @ S) @ CZ
    # Relation 3
    LHS = B02 @ np.kron(I, A10) @ CZ
    RHS = residual3 @ B01 @ np.kron(I, A10) @ wI
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(np.linalg.matrix_power(X, 2) @ S, np.linalg.matrix_power(Z, 2) 
                        @ S @ H @ np.linalg.matrix_power(S, 2) @ H) @ np.linalg.matrix_power(CZ, 2) @ np.kron(I, H @ S)
    # Relation 4
    LHS = B10 @ np.kron(I, A10) @ CZ
    RHS = residual4 @ B12 @ np.kron(I, A12) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = B11 @ np.kron(I, A10) @ CZ
    RHS = residual4 @ B10 @ np.kron(I, A12) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = B12 @ np.kron(I, A10) @ CZ
    RHS = residual4 @ B11 @ np.kron(I, A12) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(I, np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(S, 2) @ H 
                        @ np.linalg.matrix_power(S, 2)) @ np.linalg.matrix_power(CZ, 2) @ np.kron(S @ X @ S, H @ S @ np.linalg.matrix_power(H, 2))
    # Relation 7
    LHS = B20 @ np.kron(I, A10) @ CZ
    RHS = residual5 @ B22 @ np.kron(I, A11) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(I, A10) @ CZ
    RHS = residual5 @ B20 @ np.kron(I, A11) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = B22 @ np.kron(I, A10) @ CZ
    RHS = residual5 @ B21 @ np.kron(I, A11) @ wI
    equalUpToScalar(LHS, RHS)

In [137]:
CZ_A10_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [138]:
def CZ_A11_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    residual1 = np.kron(I, np.linalg.matrix_power(H, 2)) @ np.linalg.matrix_power(CZ, 2)
    # Relation 1
    LHS = B00 @ np.kron(I, A11) @ CZ
    RHS = residual1 @ B02 @ np.kron(I, A11)
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2) 
                        @ np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2)) @ CZ
    # Relation 2
    LHS = B01 @ np.kron(I, A11) @ CZ
    RHS = residual2 @ B00 @ np.kron(I, A11) @ w2I
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, S @ np.linalg.matrix_power(H, 2) @ S) @ CZ
    # Relation 3
    LHS = B02 @ np.kron(I, A11) @ CZ
    RHS = residual3 @ B01 @ np.kron(I, A11) @ wI
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(np.linalg.matrix_power(X, 2) @ S, np.linalg.matrix_power(Z, 2) 
                        @ S @ H @ np.linalg.matrix_power(S, 2) @ H) @ np.linalg.matrix_power(CZ, 2) @ np.kron(I, H @ S)
    # Relation 4
    LHS = B10 @ np.kron(I, A11) @ CZ
    RHS = residual4 @ B12 @ np.kron(I, A10) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = B11 @ np.kron(I, A11) @ CZ
    RHS = residual4 @ B10 @ np.kron(I, A10) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = B12 @ np.kron(I, A11) @ CZ
    RHS = residual4 @ B11 @ np.kron(I, A10) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(I, np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(S, 2) @ H 
                        @ np.linalg.matrix_power(S, 2)) @ np.linalg.matrix_power(CZ, 2) @ np.kron(S @ X @ S, H @ S @ np.linalg.matrix_power(H, 2))
    # Relation 7
    LHS = B20 @ np.kron(I, A11) @ CZ
    RHS = residual5 @ B22 @ np.kron(I, A12) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(I, A11) @ CZ
    RHS = residual5 @ B20 @ np.kron(I, A12) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = B22 @ np.kron(I, A11) @ CZ
    RHS = residual5 @ B21 @ np.kron(I, A12) @ wI
    equalUpToScalar(LHS, RHS)

In [139]:
CZ_A11_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [140]:
def CZ_A12_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    residual1 = np.kron(I, np.linalg.matrix_power(H, 2)) @ np.linalg.matrix_power(CZ, 2)
    # Relation 1
    LHS = B00 @ np.kron(I, A12) @ CZ
    RHS = residual1 @ B02 @ np.kron(I, A12)
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2) 
                        @ np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2)) @ CZ
    # Relation 2
    LHS = B01 @ np.kron(I, A12) @ CZ
    RHS = residual2 @ B00 @ np.kron(I, A12) @ w2I
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, S @ np.linalg.matrix_power(H, 2) @ S) @ CZ
    # Relation 3
    LHS = B02 @ np.kron(I, A12) @ CZ
    RHS = residual3 @ B01 @ np.kron(I, A12) @ wI
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(np.linalg.matrix_power(X, 2) @ S, np.linalg.matrix_power(Z, 2) 
                        @ S @ H @ np.linalg.matrix_power(S, 2) @ H) @ np.linalg.matrix_power(CZ, 2) @ np.kron(I, H @ S)
    # Relation 4
    LHS = B10 @ np.kron(I, A12) @ CZ
    RHS = residual4 @ B12 @ np.kron(I, A11) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = B11 @ np.kron(I, A12) @ CZ
    RHS = residual4 @ B10 @ np.kron(I, A11) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = B12 @ np.kron(I, A12) @ CZ
    RHS = residual4 @ B11 @ np.kron(I, A11) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(I, np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(S, 2) @ H 
                        @ np.linalg.matrix_power(S, 2)) @ np.linalg.matrix_power(CZ, 2) @ np.kron(S @ X @ S, H @ S @ np.linalg.matrix_power(H, 2))
    
    # Relation 7
    LHS = B20 @ np.kron(I, A12) @ CZ
    RHS = residual5 @ B22 @ np.kron(I, A10) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(I, A12) @ CZ
    RHS = residual5 @ B20 @ np.kron(I, A10) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = B22 @ np.kron(I, A12) @ CZ
    RHS = residual5 @ B21 @ np.kron(I, A10) @ wI
    equalUpToScalar(LHS, RHS)

In [141]:
CZ_A12_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [142]:
def CZ_A20_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    residual1 = np.kron(I, np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2) 
                        @ np.linalg.matrix_power(S, 2)) @ np.linalg.matrix_power(CZ, 2)
    # Relation 1
    LHS = B00 @ np.kron(I, A20) @ CZ
    RHS = residual1 @ B01 @ np.kron(I, A20) @ wI
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.linalg.matrix_power(S, 2) 
                        @ np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2)) @ CZ
    # Relation 2
    LHS = B01 @ np.kron(I, A20) @ CZ
    RHS = residual2 @ B02 @ np.kron(I, A20) @ w2I
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, np.linalg.matrix_power(H, 2)) @ np.linalg.matrix_power(CZ, 2)
    # Relation 3
    LHS = B02 @ np.kron(I, A20) @ CZ
    RHS = residual3 @ B00 @ np.kron(I, A20)
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(I, np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(S, 2) @ H 
                        @ np.linalg.matrix_power(S, 2)) @ np.linalg.matrix_power(CZ, 2) @ np.kron(S @ X @ S, H @ S @ np.linalg.matrix_power(H, 2))
    # Relation 4
    LHS = B10 @ np.kron(I, A20) @ CZ
    RHS = residual4 @ B11 @ np.kron(I, A22) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = B11 @ np.kron(I, A20) @ CZ
    RHS = residual4 @ B12 @ np.kron(I, A22) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = B12 @ np.kron(I, A20) @ CZ
    RHS = residual4 @ B10 @ np.kron(I, A22) @ wI
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(np.linalg.matrix_power(X, 2) @ S, np.linalg.matrix_power(Z, 2) 
                        @ S @ H @ np.linalg.matrix_power(S, 2) @ H) @ np.linalg.matrix_power(CZ, 2) @ np.kron(I, H @ S)
    # Relation 7
    LHS = B20 @ np.kron(I, A20) @ CZ
    RHS = residual5 @ B21 @ np.kron(I, A21) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(I, A20) @ CZ
    RHS = residual5 @ B22 @ np.kron(I, A21) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = B22 @ np.kron(I, A20) @ CZ
    RHS = residual5 @ B20 @ np.kron(I, A21) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

In [143]:
CZ_A20_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [144]:
def CZ_A21_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    residual1 = np.kron(I, np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2) 
                        @ np.linalg.matrix_power(S, 2)) @ np.linalg.matrix_power(CZ, 2)
    # Relation 1
    LHS = B00 @ np.kron(I, A21) @ CZ
    RHS = residual1 @ B01 @ np.kron(I, A21) @ wI
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.linalg.matrix_power(S, 2) 
                        @ np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2)) @ CZ
    # Relation 2
    LHS = B01 @ np.kron(I, A21) @ CZ
    RHS = residual2 @ B02 @ np.kron(I, A21) @ w2I
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, np.linalg.matrix_power(H, 2)) @ np.linalg.matrix_power(CZ, 2)
    # Relation 3
    LHS = B02 @ np.kron(I, A21) @ CZ
    RHS = residual3 @ B00 @ np.kron(I, A21)
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(I, np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(S, 2) @ H 
                        @ np.linalg.matrix_power(S, 2)) @ np.linalg.matrix_power(CZ, 2) @ np.kron(S @ X @ S, H @ S @ np.linalg.matrix_power(H, 2))
    # Relation 4
    LHS = B10 @ np.kron(I, A21) @ CZ
    RHS = residual4 @ B11 @ np.kron(I, A20) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = B11 @ np.kron(I, A21) @ CZ
    RHS = residual4 @ B12 @ np.kron(I, A20) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = B12 @ np.kron(I, A21) @ CZ
    RHS = residual4 @ B10 @ np.kron(I, A20) @ wI
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(np.linalg.matrix_power(X, 2) @ S, np.linalg.matrix_power(Z, 2) 
                        @ S @ H @ np.linalg.matrix_power(S, 2) @ H) @ np.linalg.matrix_power(CZ, 2) @ np.kron(I, H @ S)
    # Relation 7
    LHS = B20 @ np.kron(I, A21) @ CZ
    RHS = residual5 @ B21 @ np.kron(I, A22) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(I, A21) @ CZ
    RHS = residual5 @ B22 @ np.kron(I, A22) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = B22 @ np.kron(I, A21) @ CZ
    RHS = residual5 @ B20 @ np.kron(I, A22) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

In [145]:
CZ_A21_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [146]:
def CZ_A22_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    residual1 = np.kron(I, np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2) @ S @ np.linalg.matrix_power(H, 2) 
                        @ np.linalg.matrix_power(S, 2)) @ np.linalg.matrix_power(CZ, 2)
    # Relation 1
    LHS = B00 @ np.kron(I, A22) @ CZ
    RHS = residual1 @ B01 @ np.kron(I, A22) @ wI
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.linalg.matrix_power(S, 2) 
                        @ np.linalg.matrix_power(H, 2) @ np.linalg.matrix_power(S, 2)) @ CZ
    # Relation 2
    LHS = B01 @ np.kron(I, A22) @ CZ
    RHS = residual2 @ B02 @ np.kron(I, A22) @ w2I
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, np.linalg.matrix_power(H, 2)) @ np.linalg.matrix_power(CZ, 2)
    # Relation 3
    LHS = B02 @ np.kron(I, A22) @ CZ
    RHS = residual3 @ B00 @ np.kron(I, A22)
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(I, np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(S, 2) @ H 
                        @ np.linalg.matrix_power(S, 2)) @ np.linalg.matrix_power(CZ, 2) @ np.kron(S @ X @ S, H @ S @ np.linalg.matrix_power(H, 2))
    # Relation 4
    LHS = B10 @ np.kron(I, A22) @ CZ
    RHS = residual4 @ B11 @ np.kron(I, A21) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = B11 @ np.kron(I, A22) @ CZ
    RHS = residual4 @ B12 @ np.kron(I, A21) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = B12 @ np.kron(I, A22) @ CZ
    RHS = residual4 @ B10 @ np.kron(I, A21) @ wI
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(np.linalg.matrix_power(X, 2) @ S, np.linalg.matrix_power(Z, 2) 
                        @ S @ H @ np.linalg.matrix_power(S, 2) @ H) @ np.linalg.matrix_power(CZ, 2) @ np.kron(I, H @ S)
    # Relation 7
    LHS = B20 @ np.kron(I, A22) @ CZ
    RHS = residual5 @ B21 @ np.kron(I, A20) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(I, A22) @ CZ
    RHS = residual5 @ B22 @ np.kron(I, A20) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = B22 @ np.kron(I, A22) @ CZ
    RHS = residual5 @ B20 @ np.kron(I, A20) * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

In [147]:
CZ_A22_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [148]:
def HxI_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    # Relation 1
    LHS = B00 @ np.kron(H, I) 
    RHS = CZ @ np.kron(I, H) @ np.linalg.matrix_power(CZ, 2) @ B00
    equalUpToScalar(LHS, RHS)

    residual1 = np.kron(I,np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(S, 2)) @ CZ
    
    # Relation 2
    LHS = B01 @ np.kron(H, I) 
    RHS = residual1 @ B10 @ wI
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, Z @ S) @ np.linalg.matrix_power(CZ, 2)
    
    # Relation 3
    LHS = B02 @ np.kron(H, I) 
    RHS = residual2 @ B20 @ w2I
    equalUpToScalar(LHS, RHS)
    
    # Relation 4
    LHS = B10 @ np.kron(H, I) 
    RHS = residual1 @ B02 @ wI
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(X, 2), Z 
                  @ np.linalg.matrix_power(X, 2) @ np.linalg.matrix_power(S, 2))
    
    # Relation 5
    LHS = B11 @ np.kron(H, I) 
    RHS = residual3 @ B12 * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(Z @ X, Z @ X @ S)
    
    # Relation 6
    LHS = B12 @ np.kron(H, I) 
    RHS = residual4 @ B22 * (-1) @ w2I
    equalUpToScalar(LHS, RHS)
    
    # Relation 7
    LHS = B20 @ np.kron(H, I) 
    RHS = residual2 @ B01 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(H, I) 
    RHS = residual4 @ B11 * (-1) @ w2I
    equalUpToScalar(LHS, RHS)
    
    # Relation 9
    LHS = B22 @ np.kron(H, I) 
    RHS = residual3 @ B21 * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

In [149]:
HxI_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [150]:
def SxI_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    # Relation 1
    LHS = B00 @ np.kron(S, I) 
    RHS = np.kron(I, S) @ B00
    equalUpToScalar(LHS, RHS)
    
    # Relation 2
    LHS = B01 @ np.kron(S, I) 
    RHS = np.kron(I, S) @ B01
    equalUpToScalar(LHS, RHS)

    residual1 = np.kron(I, np.linalg.matrix_power(Z, 2) @ S)
    
    # Relation 3
    LHS = B02 @ np.kron(S, I) 
    RHS = residual1 @ B02
    equalUpToScalar(LHS, RHS)

    # Relation 4
    LHS = B10 @ np.kron(S, I) 
    RHS = B11
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = B11 @ np.kron(S, I) 
    RHS = B12
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = B12 @ np.kron(S, I) 
    RHS = B10
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(Z @ X ,np.linalg.matrix_power(Z, 2) @ X)

    # Relation 7
    LHS = B20 @ np.kron(S, I) 
    RHS = residual2 @ B22
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(S, I) 
    RHS = residual2 @ B20
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = B22 @ np.kron(S, I) 
    RHS = residual2 @ B21
    equalUpToScalar(LHS, RHS)

In [151]:
SxI_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [152]:
def CZ_B00_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    # Relation 1
    LHS = np.kron(B00, I) @ np.kron(I, B00) @ np.kron(CZ, I)
    RHS = np.kron(I, CZ) @ np.kron(B00, I) @ np.kron(I, B00)
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(B01, I) @ np.kron(I, B00) @ np.kron(CZ, I)
    RHS = np.kron(I, CZ) @ np.kron(B01, I) @ np.kron(I, B00)
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(B02, I) @ np.kron(I, B00) @ np.kron(CZ, I)
    RHS = np.kron(I, np.linalg.matrix_power(CZ, 2)) @ np.kron(B02, I) @ np.kron(I, B00)
    equalUpToScalar(LHS, RHS)

    residual1 = np.kron(I, np.kron(np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H, I) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(H @ S @ H, np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2)))

    # Relation 4
    LHS = np.kron(B10, I) @ np.kron(I, B00) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B10, I) @ np.kron(I, B02) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(B11, I) @ np.kron(I, B00) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B11, I) @ np.kron(I, B02) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(B12, I) @ np.kron(I, B00) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B12, I) @ np.kron(I, B02) @ wI
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.kron(H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3), I) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(H @ S @ np.linalg.matrix_power(H, 3), np.linalg.matrix_power(S, 2)))

    # Relation 7
    LHS = np.kron(B20, I) @ np.kron(I, B00) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B20, I) @ np.kron(I, B01) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(B21, I) @ np.kron(I, B00) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B21, I) @ np.kron(I, B01) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(B22, I) @ np.kron(I, B00) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B22, I) @ np.kron(I, B01) @ wI
    equalUpToScalar(LHS, RHS)

In [153]:
CZ_B00_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [154]:
def CZ_B01_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    # Relation 1
    LHS = np.kron(B00, I) @ np.kron(I, B01) @ np.kron(CZ, I)
    RHS = np.kron(I, CZ) @ np.kron(B00, I) @ np.kron(I, B01)
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(B01, I) @ np.kron(I, B01) @ np.kron(CZ, I)
    RHS = np.kron(I, CZ) @ np.kron(B01, I) @ np.kron(I, B01)
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(B02, I) @ np.kron(I, B01) @ np.kron(CZ, I)
    RHS = np.kron(I, np.linalg.matrix_power(CZ, 2)) @ np.kron(B02, I) @ np.kron(I, B01)
    equalUpToScalar(LHS, RHS)

    residual1 = np.kron(I, np.kron(S @ np.linalg.matrix_power(H, 3), I) @ np.linalg.matrix_power(CZ, 2) 
                        @ np.kron(H @ np.linalg.matrix_power(S, 2), S))

    # Relation 4
    LHS = np.kron(B10, I) @ np.kron(I, B01) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B10, I) @ np.kron(I, B00) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(B11, I) @ np.kron(I, B01) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B11, I) @ np.kron(I, B00) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(B12, I) @ np.kron(I, B01) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B12, I) @ np.kron(I, B00) @ w2I
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.kron(X @ np.linalg.matrix_power(Z, 2) @ S @ H @ S, Z @ np.linalg.matrix_power(H, 2))
                       @ CZ @ np.kron(H @ S @ np.linalg.matrix_power(H, 3), I))

    # Relation 7
    LHS = np.kron(B20, I) @ np.kron(I, B01) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B20, I) @ np.kron(I, B02) * (-1) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(B21, I) @ np.kron(I, B01) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B21, I) @ np.kron(I, B02) * (-1) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(B22, I) @ np.kron(I, B01) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B22, I) @ np.kron(I, B02) * (-1) @ wI
    equalUpToScalar(LHS, RHS)

In [155]:
CZ_B01_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [156]:
def CZ_B02_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    # Relation 1
    LHS = np.kron(B00, I) @ np.kron(I, B02) @ np.kron(CZ, I)
    RHS = np.kron(I, np.linalg.matrix_power(CZ, 2)) @ np.kron(B00, I) @ np.kron(I, B02)
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(B01, I) @ np.kron(I, B02) @ np.kron(CZ, I)
    RHS = np.kron(I, np.linalg.matrix_power(CZ, 2)) @ np.kron(B01, I) @ np.kron(I, B02)
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(B02, I) @ np.kron(I, B02) @ np.kron(CZ, I)
    RHS = np.kron(I, CZ) @ np.kron(B02, I) @ np.kron(I, B02)
    equalUpToScalar(LHS, RHS)

    residual1 = np.kron(I, np.kron(X @ np.linalg.matrix_power(Z, 2) @ S @ H @ S, Z @ np.linalg.matrix_power(H, 2))
                       @ CZ @ np.kron(H @ S @ np.linalg.matrix_power(H, 3), I))

    # Relation 4
    LHS = np.kron(B10, I) @ np.kron(I, B02) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B10, I) @ np.kron(I, B01) * (-1) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(B11, I) @ np.kron(I, B02) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B11, I) @ np.kron(I, B01) * (-1) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(B12, I) @ np.kron(I, B02) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B12, I) @ np.kron(I, B01) * (-1) @ wI
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.kron(S @ np.linalg.matrix_power(H, 3), S @ np.linalg.matrix_power(H, 2)) @ CZ 
                        @ np.kron(H @ np.linalg.matrix_power(S, 2), I))

    # Relation 7
    LHS = np.kron(B20, I) @ np.kron(I, B02) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B20, I) @ np.kron(I, B00) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(B21, I) @ np.kron(I, B02) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B21, I) @ np.kron(I, B00) @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(B22, I) @ np.kron(I, B02) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B22, I) @ np.kron(I, B00) @ w2I
    equalUpToScalar(LHS, RHS)

In [157]:
CZ_B02_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [158]:
def CZ_B10_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(I, np.kron(I, S @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(S @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 1
    LHS = np.kron(B00, I) @ np.kron(I, B10) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B02, I) @ np.kron(I, B10) @ w2I
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.kron(I, S @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(np.linalg.matrix_power(Z, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 2
    LHS = np.kron(B01, I) @ np.kron(I, B10) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B00, I) @ np.kron(I, B10)
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, np.kron(Z, np.linalg.matrix_power(Z, 2) @ S @ H) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2)))

    # Relation 3
    LHS = np.kron(B02, I) @ np.kron(I, B10) @ np.kron(CZ, I)
    RHS = residual3 @ np.kron(B01, I) @ np.kron(I, B10) @ wI
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(X, 2), 
                        np.kron(Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H, Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H) 
                        @ CZ @ np.kron(H @ np.linalg.matrix_power(S, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 4
    LHS = np.kron(B10, I) @ np.kron(I, B10) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B12, I) @ np.kron(I, B12) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(B11, I) @ np.kron(I, B10) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B10, I) @ np.kron(I, B12) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(B12, I) @ np.kron(I, B10) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B11, I) @ np.kron(I, B12) @ wI
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(Z @ X, np.kron(np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2), 
                                       np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2)) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), 
                                                                  H @ np.linalg.matrix_power(S, 2)@ np.linalg.matrix_power(H, 2)))
    # Relation 7
    LHS = np.kron(B20, I) @ np.kron(I, B10) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B22, I) @ np.kron(I, B11)
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(B21, I) @ np.kron(I, B10) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B20, I) @ np.kron(I, B11)
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(B22, I) @ np.kron(I, B10) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B21, I) @ np.kron(I, B11)
    equalUpToScalar(LHS, RHS)

In [159]:
CZ_B10_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [160]:
def CZ_B11_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(I, np.kron(I, S @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(S @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 1
    LHS = np.kron(B00, I) @ np.kron(I, B11) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B02, I) @ np.kron(I, B11) @ w2I
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.kron(I, S @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(np.linalg.matrix_power(Z, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 2
    LHS = np.kron(B01, I) @ np.kron(I, B11) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B00, I) @ np.kron(I, B11)
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, np.kron(Z, np.linalg.matrix_power(Z, 2) @ S @ H) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2)))

    # Relation 3
    LHS = np.kron(B02, I) @ np.kron(I, B11) @ np.kron(CZ, I)
    RHS = residual3 @ np.kron(B01, I) @ np.kron(I, B11) @ wI
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(X, 2), 
                        np.kron(Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H, Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H) 
                        @ CZ @ np.kron(H @ np.linalg.matrix_power(S, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 4
    LHS = np.kron(B10, I) @ np.kron(I, B11) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B12, I) @ np.kron(I, B10) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(B11, I) @ np.kron(I, B11) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B10, I) @ np.kron(I, B10) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(B12, I) @ np.kron(I, B11) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B11, I) @ np.kron(I, B10) @ wI
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(Z @ X, np.kron(np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2), 
                                       np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2)) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), 
                                                                  H @ np.linalg.matrix_power(S, 2)@ np.linalg.matrix_power(H, 2)))
    # Relation 7
    LHS = np.kron(B20, I) @ np.kron(I, B11) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B22, I) @ np.kron(I, B12)
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(B21, I) @ np.kron(I, B11) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B20, I) @ np.kron(I, B12)
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(B22, I) @ np.kron(I, B11) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B21, I) @ np.kron(I, B12)
    equalUpToScalar(LHS, RHS)

In [161]:
CZ_B11_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [162]:
def CZ_B12_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(I, np.kron(I, S @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(S @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 1
    LHS = np.kron(B00, I) @ np.kron(I, B12) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B02, I) @ np.kron(I, B12) @ w2I
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.kron(I, S @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(np.linalg.matrix_power(Z, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 2
    LHS = np.kron(B01, I) @ np.kron(I, B12) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B00, I) @ np.kron(I, B12)
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, np.kron(Z, np.linalg.matrix_power(Z, 2) @ S @ H) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2)))

    # Relation 3
    LHS = np.kron(B02, I) @ np.kron(I, B12) @ np.kron(CZ, I)
    RHS = residual3 @ np.kron(B01, I) @ np.kron(I, B12) @ wI
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(X, 2), 
                        np.kron(Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H, Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H) 
                        @ CZ @ np.kron(H @ np.linalg.matrix_power(S, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 4
    LHS = np.kron(B10, I) @ np.kron(I, B12) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B12, I) @ np.kron(I, B11) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(B11, I) @ np.kron(I, B12) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B10, I) @ np.kron(I, B11) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(B12, I) @ np.kron(I, B12) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B11, I) @ np.kron(I, B11) @ wI
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(Z @ X, np.kron(np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2), 
                                       np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2)) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), 
                                                                  H @ np.linalg.matrix_power(S, 2)@ np.linalg.matrix_power(H, 2)))
    # Relation 7
    LHS = np.kron(B20, I) @ np.kron(I, B12) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B22, I) @ np.kron(I, B10)
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(B21, I) @ np.kron(I, B12) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B20, I) @ np.kron(I, B10)
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(B22, I) @ np.kron(I, B12) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B21, I) @ np.kron(I, B10)
    equalUpToScalar(LHS, RHS)

In [163]:
CZ_B12_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [164]:
def CZ_B20_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(I, np.kron(I, Z @ S @ H) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(I, H @ S @ np.linalg.matrix_power(H, 2) @ S))

    # Relation 1
    LHS = np.kron(B00, I) @ np.kron(I, B20) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B01, I) @ np.kron(I, B20)
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.kron(I, np.linalg.matrix_power(Z, 2) @ S @ H) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(S @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2)))

    # Relation 2
    LHS = np.kron(B01, I) @ np.kron(I, B20) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B02, I) @ np.kron(I, B20) @ w2I
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, np.kron(Z, S @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 3
    LHS = np.kron(B02, I) @ np.kron(I, B20) @ np.kron(CZ, I)
    RHS = residual3 @ np.kron(B00, I) @ np.kron(I, B20) @ wI
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(Z @ X, np.kron(np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2), 
                                       np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2)) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), 
                                                                  H @ np.linalg.matrix_power(S, 2)@ np.linalg.matrix_power(H, 2)))
    
    # Relation 4
    LHS = np.kron(B10, I) @ np.kron(I, B20) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B11, I) @ np.kron(I, B22)
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(B11, I) @ np.kron(I, B20) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B12, I) @ np.kron(I, B22)
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(B12, I) @ np.kron(I, B20) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B10, I) @ np.kron(I, B22)
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(X, 2), 
                        np.kron(Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H, 
                                Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H) 
                        @ CZ @ np.kron(H @ np.linalg.matrix_power(S, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 7
    LHS = np.kron(B20, I) @ np.kron(I, B20) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B21, I) @ np.kron(I, B21) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(B21, I) @ np.kron(I, B20) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B22, I) @ np.kron(I, B21) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(B22, I) @ np.kron(I, B20) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B20, I) @ np.kron(I, B21) @ wI
    equalUpToScalar(LHS, RHS)

In [165]:
CZ_B20_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [166]:
def CZ_B21_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(I, np.kron(I, Z @ S @ H) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(I, H @ S @ np.linalg.matrix_power(H, 2) @ S))

    # Relation 1
    LHS = np.kron(B00, I) @ np.kron(I, B21) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B01, I) @ np.kron(I, B21)
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.kron(I, np.linalg.matrix_power(Z, 2) @ S @ H) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(S @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2)))

    # Relation 2
    LHS = np.kron(B01, I) @ np.kron(I, B21) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B02, I) @ np.kron(I, B21) @ w2I
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, np.kron(Z, S @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 3
    LHS = np.kron(B02, I) @ np.kron(I, B21) @ np.kron(CZ, I)
    RHS = residual3 @ np.kron(B00, I) @ np.kron(I, B21) @ wI
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(Z @ X, np.kron(np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2), 
                                       np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2)) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), 
                                                                  H @ np.linalg.matrix_power(S, 2)@ np.linalg.matrix_power(H, 2)))
    
    # Relation 4
    LHS = np.kron(B10, I) @ np.kron(I, B21) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B11, I) @ np.kron(I, B20)
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(B11, I) @ np.kron(I, B21) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B12, I) @ np.kron(I, B20)
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(B12, I) @ np.kron(I, B21) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B10, I) @ np.kron(I, B20)
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(X, 2), 
                        np.kron(Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H, 
                                Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H) 
                        @ CZ @ np.kron(H @ np.linalg.matrix_power(S, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 7
    LHS = np.kron(B20, I) @ np.kron(I, B21) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B21, I) @ np.kron(I, B22) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(B21, I) @ np.kron(I, B21) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B22, I) @ np.kron(I, B22) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(B22, I) @ np.kron(I, B21) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B20, I) @ np.kron(I, B22) @ wI
    equalUpToScalar(LHS, RHS)

In [167]:
CZ_B21_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [168]:
def CZ_B22_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(I, np.kron(I, Z @ S @ H) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(I, H @ S @ np.linalg.matrix_power(H, 2) @ S))

    # Relation 1
    LHS = np.kron(B00, I) @ np.kron(I, B22) @ np.kron(CZ, I)
    RHS = residual1 @ np.kron(B01, I) @ np.kron(I, B22)
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.kron(I, np.linalg.matrix_power(Z, 2) @ S @ H) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(S @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2)))

    # Relation 2
    LHS = np.kron(B01, I) @ np.kron(I, B22) @ np.kron(CZ, I)
    RHS = residual2 @ np.kron(B02, I) @ np.kron(I, B22) @ w2I
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(I, np.kron(Z, S @ np.linalg.matrix_power(H, 3)) @ np.linalg.matrix_power(CZ, 2)
                       @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 3
    LHS = np.kron(B02, I) @ np.kron(I, B22) @ np.kron(CZ, I)
    RHS = residual3 @ np.kron(B00, I) @ np.kron(I, B22) @ wI
    equalUpToScalar(LHS, RHS)

    residual4 = np.kron(Z @ X, np.kron(np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2), 
                                       np.linalg.matrix_power(Z, 2) @ S @ H @ np.linalg.matrix_power(S, 2)) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 2), 
                                                                  H @ np.linalg.matrix_power(S, 2)@ np.linalg.matrix_power(H, 2)))
    
    # Relation 4
    LHS = np.kron(B10, I) @ np.kron(I, B22) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B11, I) @ np.kron(I, B21)
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(B11, I) @ np.kron(I, B22) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B12, I) @ np.kron(I, B21)
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(B12, I) @ np.kron(I, B22) @ np.kron(CZ, I)
    RHS = residual4 @ np.kron(B10, I) @ np.kron(I, B21)
    equalUpToScalar(LHS, RHS)

    residual5 = np.kron(np.linalg.matrix_power(Z, 2) @ np.linalg.matrix_power(X, 2), 
                        np.kron(Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H, 
                                Z @ np.linalg.matrix_power(X, 2) @ H @ np.linalg.matrix_power(S, 2) @ H) 
                        @ CZ @ np.kron(H @ np.linalg.matrix_power(S, 2), H @ np.linalg.matrix_power(S, 2)))

    # Relation 7
    LHS = np.kron(B20, I) @ np.kron(I, B22) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B21, I) @ np.kron(I, B20) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(B21, I) @ np.kron(I, B22) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B22, I) @ np.kron(I, B20) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(B22, I) @ np.kron(I, B22) @ np.kron(CZ, I)
    RHS = residual5 @ np.kron(B20, I) @ np.kron(I, B20) @ wI
    equalUpToScalar(LHS, RHS)

In [169]:
CZ_B22_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [170]:
def IxH_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    # Relation 1
    LHS = D00 @ np.kron(I, H) 
    RHS = np.kron(H, I) @ D00
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = D01 @ np.kron(I, H) 
    RHS = D10
    equalUpToScalar(LHS, RHS)

    residual0 = np.kron(np.linalg.matrix_power(H, 2), I)

    # Relation 3
    LHS = D02 @ np.kron(I, H) 
    RHS = residual0 @ D20
    equalUpToScalar(LHS, RHS)

    # Relation 4
    LHS = D10 @ np.kron(I, H) 
    RHS = residual0 @ D02
    equalUpToScalar(LHS, RHS)

    residual1 = np.kron(np.linalg.matrix_power(X, 2) @ np.linalg.matrix_power(S, 2), Z)

    # Relation 5
    LHS = D11 @ np.kron(I, H) 
    RHS = residual1 @ D12 * (-1) @ w2I
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(X @ S @ np.linalg.matrix_power(Z, 2), np.linalg.matrix_power(Z, 2))

    # Relation 6
    LHS = D12 @ np.kron(I, H) 
    RHS = residual2 @ D22 * (-1) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 7
    LHS = D20 @ np.kron(I, H) 
    RHS = D01
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = D21 @ np.kron(I, H) 
    RHS = residual2 @ D11 * (-1) @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = D22 @ np.kron(I, H) 
    RHS = residual1 @ D21 * (-1) @ w2I
    equalUpToScalar(LHS, RHS) 

In [171]:
IxH_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [172]:
def IxS_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    # Relation 1
    LHS = D00 @ np.kron(I, S) 
    RHS = np.kron(S, I) @ D00
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = D01 @ np.kron(I, S) 
    RHS = np.kron(S, I) @ D01
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = D02 @ np.kron(I, S) 
    RHS = np.kron(S, I) @ D02
    equalUpToScalar(LHS, RHS)

    # Relation 4
    LHS = D10 @ np.kron(I, S) 
    RHS = D11
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = D11 @ np.kron(I, S) 
    RHS = D12
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = D12 @ np.kron(I, S) 
    RHS = D10
    equalUpToScalar(LHS, RHS)

    residual = np.kron(X, Z @ Z)

    # Relation 7
    LHS = D20 @ np.kron(I, S) 
    RHS = residual @ D22
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = D21 @ np.kron(I, S) 
    RHS = residual @ D20
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = D22 @ np.kron(I, S) 
    RHS = residual @ D21
    equalUpToScalar(LHS, RHS) 

In [173]:
IxS_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [174]:
def CZ_D00_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    # Relation 1
    LHS = np.kron(I, D00)@ np.kron(D00, I) @ np.kron(I, CZ)
    RHS = np.kron(CZ, I) @ np.kron(I, D00) @ np.kron(D00, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(I, D01)@ np.kron(D00, I) @ np.kron(I, CZ)
    RHS = np.kron(CZ, I) @ np.kron(I, D01) @ np.kron(D00, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(I, D02)@ np.kron(D00, I) @ np.kron(I, CZ)
    RHS = np.kron(CZ, I) @ np.kron(I, D02) @ np.kron(D00, I) 
    equalUpToScalar(LHS, RHS)

    residual1 = np.kron(np.kron(I,np.linalg.matrix_power(H, 3)) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(I,H), I)

    # Relation 4
    LHS = np.kron(I, D10)@ np.kron(D00, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D10) @ np.kron(D02, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(I, D11)@ np.kron(D00, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D11) @ np.kron(D02, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(I, D12)@ np.kron(D00, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D12) @ np.kron(D02, I) 
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(np.kron(I,np.linalg.matrix_power(H, 3)) @ CZ @ np.kron(I,H), I)

    # Relation 7
    LHS = np.kron(I, D20)@ np.kron(D00, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D20) @ np.kron(D01, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(I, D21)@ np.kron(D00, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D21) @ np.kron(D01, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(I, D22)@ np.kron(D00, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D22) @ np.kron(D01, I) 
    equalUpToScalar(LHS, RHS)

In [175]:
CZ_D00_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [176]:
def CZ_D01_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    # Relation 1
    LHS = np.kron(I, D00)@ np.kron(D01, I) @ np.kron(I, CZ)
    RHS = np.kron(CZ, I) @ np.kron(I, D00) @ np.kron(D01, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(I, D01)@ np.kron(D01, I) @ np.kron(I, CZ)
    RHS = np.kron(CZ, I) @ np.kron(I, D01) @ np.kron(D01, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(I, D02)@ np.kron(D01, I) @ np.kron(I, CZ)
    RHS = np.kron(CZ, I) @ np.kron(I, D02) @ np.kron(D01, I) 
    equalUpToScalar(LHS, RHS)

    residual1 = np.kron(np.kron(I,np.linalg.matrix_power(H, 3)) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(I,H), I)

    # Relation 4
    LHS = np.kron(I, D10)@ np.kron(D01, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D10) @ np.kron(D00, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(I, D11)@ np.kron(D01, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D11) @ np.kron(D00, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(I, D12)@ np.kron(D01, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D12) @ np.kron(D00, I) 
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(np.kron(I,np.linalg.matrix_power(H, 3)) @ CZ @ np.kron(I,H), I)

    # Relation 7
    LHS = np.kron(I, D20)@ np.kron(D01, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D20) @ np.kron(D02, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(I, D21)@ np.kron(D01, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D21) @ np.kron(D02, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(I, D22)@ np.kron(D01, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D22) @ np.kron(D02, I) 
    equalUpToScalar(LHS, RHS)

In [177]:
CZ_D01_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [178]:
def CZ_D02_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    # Relation 1
    LHS = np.kron(I, D00)@ np.kron(D02, I) @ np.kron(I, CZ)
    RHS = np.kron(CZ, I) @ np.kron(I, D00) @ np.kron(D02, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(I, D01)@ np.kron(D02, I) @ np.kron(I, CZ)
    RHS = np.kron(CZ, I) @ np.kron(I, D01) @ np.kron(D02, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(I, D02)@ np.kron(D02, I) @ np.kron(I, CZ)
    RHS = np.kron(CZ, I) @ np.kron(I, D02) @ np.kron(D02, I) 
    equalUpToScalar(LHS, RHS)

    residual1 = np.kron(np.kron(I,np.linalg.matrix_power(H, 3)) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(I,H), I)

    # Relation 4
    LHS = np.kron(I, D10)@ np.kron(D02, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D10) @ np.kron(D01, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(I, D11)@ np.kron(D02, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D11) @ np.kron(D01, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(I, D12)@ np.kron(D02, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D12) @ np.kron(D01, I) 
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(np.kron(I,np.linalg.matrix_power(H, 3)) @ CZ @ np.kron(I,H), I)

    # Relation 7
    LHS = np.kron(I, D20)@ np.kron(D02, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D20) @ np.kron(D00, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(I, D21)@ np.kron(D02, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D21) @ np.kron(D00, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(I, D22)@ np.kron(D02, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D22) @ np.kron(D00, I) 
    equalUpToScalar(LHS, RHS)

In [179]:
CZ_D02_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [180]:
def CZ_D10_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(np.kron(np.linalg.matrix_power(H, 3), I) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(H, I), I)

    # Relation 1
    LHS = np.kron(I, D00)@ np.kron(D10, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D02) @ np.kron(D10, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(I, D01)@ np.kron(D10, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D00) @ np.kron(D10, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(I, D02)@ np.kron(D10, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D01) @ np.kron(D10, I) 
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(np.kron(H, H) @ CZ @ np.kron(S @ np.linalg.matrix_power(H, 3), 
                                                     S @ np.linalg.matrix_power(H, 3)), Z)

    # Relation 4
    LHS = np.kron(I, D10)@ np.kron(D10, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D12) @ np.kron(D12, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(I, D11)@ np.kron(D10, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D10) @ np.kron(D12, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(I, D12)@ np.kron(D10, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D11) @ np.kron(D12, I) 
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(np.kron(H, H) @ np.linalg.matrix_power(CZ, 2) 
                @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3), 
                np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3)), np.linalg.matrix_power(Z, 2))

    # Relation 7
    LHS = np.kron(I, D20)@ np.kron(D10, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D22) @ np.kron(D11, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(I, D21)@ np.kron(D10, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D20) @ np.kron(D11, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(I, D22)@ np.kron(D10, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D21) @ np.kron(D11, I) 
    equalUpToScalar(LHS, RHS)

In [181]:
CZ_D10_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [182]:
def CZ_D11_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(np.kron(np.linalg.matrix_power(H, 3), I) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(H, I), I)

    # Relation 1
    LHS = np.kron(I, D00)@ np.kron(D11, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D02) @ np.kron(D11, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(I, D01)@ np.kron(D11, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D00) @ np.kron(D11, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(I, D02)@ np.kron(D11, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D01) @ np.kron(D11, I) 
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(np.kron(H, H) @ CZ @ np.kron(S @ np.linalg.matrix_power(H, 3), 
                                                     S @ np.linalg.matrix_power(H, 3)), Z)

    # Relation 4
    LHS = np.kron(I, D10)@ np.kron(D11, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D12) @ np.kron(D10, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(I, D11)@ np.kron(D11, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D10) @ np.kron(D10, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(I, D12)@ np.kron(D11, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D11) @ np.kron(D10, I) 
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(np.kron(H, H) @ np.linalg.matrix_power(CZ, 2) 
                @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3), 
                np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3)), np.linalg.matrix_power(Z, 2))

    # Relation 7
    LHS = np.kron(I, D20)@ np.kron(D11, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D22) @ np.kron(D12, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(I, D21)@ np.kron(D11, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D20) @ np.kron(D12, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(I, D22)@ np.kron(D11, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D21) @ np.kron(D12, I) 
    equalUpToScalar(LHS, RHS)

In [183]:
CZ_D11_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [184]:
def CZ_D12_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(np.kron(np.linalg.matrix_power(H, 3), I) 
                        @ np.linalg.matrix_power(CZ, 2) @ np.kron(H, I), I)

    # Relation 1
    LHS = np.kron(I, D00)@ np.kron(D12, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D02) @ np.kron(D12, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(I, D01)@ np.kron(D12, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D00) @ np.kron(D12, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(I, D02)@ np.kron(D12, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D01) @ np.kron(D12, I) 
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(np.kron(H, H) @ CZ @ np.kron(S @ np.linalg.matrix_power(H, 3), 
                                                     S @ np.linalg.matrix_power(H, 3)), Z)

    # Relation 4
    LHS = np.kron(I, D10)@ np.kron(D12, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D12) @ np.kron(D11, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(I, D11)@ np.kron(D12, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D10) @ np.kron(D11, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(I, D12)@ np.kron(D12, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D11) @ np.kron(D11, I) 
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(np.kron(H, H) @ np.linalg.matrix_power(CZ, 2) 
                @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3), 
                np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3)), np.linalg.matrix_power(Z, 2))

    # Relation 7
    LHS = np.kron(I, D20)@ np.kron(D12, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D22) @ np.kron(D10, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(I, D21)@ np.kron(D12, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D20) @ np.kron(D10, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(I, D22)@ np.kron(D12, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D21) @ np.kron(D10, I) 
    equalUpToScalar(LHS, RHS)

In [185]:
CZ_D12_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [186]:
def CZ_D20_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(np.kron(np.linalg.matrix_power(H, 3), I) @ CZ @ np.kron(H, I), I)

    # Relation 1
    LHS = np.kron(I, D00)@ np.kron(D20, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D01) @ np.kron(D20, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(I, D01)@ np.kron(D20, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D02) @ np.kron(D20, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(I, D02)@ np.kron(D20, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D00) @ np.kron(D20, I) 
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(np.kron(H, H) @ np.linalg.matrix_power(CZ, 2) 
                @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3), 
                np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3)), np.linalg.matrix_power(Z, 2))

    # Relation 4
    LHS = np.kron(I, D10)@ np.kron(D20, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D11) @ np.kron(D22, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(I, D11)@ np.kron(D20, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D12) @ np.kron(D22, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(I, D12)@ np.kron(D20, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D10) @ np.kron(D22, I) 
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(np.kron(H, H) @ CZ @ np.kron(S @ np.linalg.matrix_power(H, 3), 
                                                     S @ np.linalg.matrix_power(H, 3)), Z)

    # Relation 7
    LHS = np.kron(I, D20)@ np.kron(D20, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D21) @ np.kron(D21, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(I, D21)@ np.kron(D20, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D22) @ np.kron(D21, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(I, D22)@ np.kron(D20, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D20) @ np.kron(D21, I) 
    equalUpToScalar(LHS, RHS)

In [187]:
CZ_D20_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [188]:
def CZ_D21_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(np.kron(np.linalg.matrix_power(H, 3), I) @ CZ @ np.kron(H, I), I)

    # Relation 1
    LHS = np.kron(I, D00)@ np.kron(D21, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D01) @ np.kron(D21, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(I, D01)@ np.kron(D21, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D02) @ np.kron(D21, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(I, D02)@ np.kron(D21, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D00) @ np.kron(D21, I) 
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(np.kron(H, H) @ np.linalg.matrix_power(CZ, 2) 
                @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3), 
                np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3)), np.linalg.matrix_power(Z, 2))

    # Relation 4
    LHS = np.kron(I, D10)@ np.kron(D21, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D11) @ np.kron(D20, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(I, D11)@ np.kron(D21, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D12) @ np.kron(D20, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(I, D12)@ np.kron(D21, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D10) @ np.kron(D20, I) 
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(np.kron(H, H) @ CZ @ np.kron(S @ np.linalg.matrix_power(H, 3), 
                                                     S @ np.linalg.matrix_power(H, 3)), Z)

    # Relation 7
    LHS = np.kron(I, D20)@ np.kron(D21, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D21) @ np.kron(D22, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(I, D21)@ np.kron(D21, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D22) @ np.kron(D22, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(I, D22)@ np.kron(D21, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D20) @ np.kron(D22, I) 
    equalUpToScalar(LHS, RHS)

In [189]:
CZ_D21_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [190]:
def CZ_D22_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(np.kron(np.linalg.matrix_power(H, 3), I) @ CZ @ np.kron(H, I), I)

    # Relation 1
    LHS = np.kron(I, D00)@ np.kron(D22, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D01) @ np.kron(D22, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(I, D01)@ np.kron(D22, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D02) @ np.kron(D22, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(I, D02)@ np.kron(D22, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(I, D00) @ np.kron(D22, I) 
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(np.kron(H, H) @ np.linalg.matrix_power(CZ, 2) 
                @ np.kron(np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3), 
                np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3)), np.linalg.matrix_power(Z, 2))

    # Relation 4
    LHS = np.kron(I, D10)@ np.kron(D22, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D11) @ np.kron(D21, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(I, D11)@ np.kron(D22, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D12) @ np.kron(D21, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(I, D12)@ np.kron(D22, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(I, D10) @ np.kron(D21, I) 
    equalUpToScalar(LHS, RHS)

    residual3 = np.kron(np.kron(H, H) @ CZ @ np.kron(S @ np.linalg.matrix_power(H, 3), 
                                                     S @ np.linalg.matrix_power(H, 3)), Z)

    # Relation 7
    LHS = np.kron(I, D20)@ np.kron(D22, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D21) @ np.kron(D20, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(I, D21)@ np.kron(D22, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D22) @ np.kron(D20, I) 
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(I, D22)@ np.kron(D22, I) @ np.kron(I, CZ)
    RHS = residual3 @ np.kron(I, D20) @ np.kron(D20, I) 
    equalUpToScalar(LHS, RHS)

In [191]:
CZ_D22_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [192]:
def IxS_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    # Relation 1
    LHS = B00 @ np.kron(I, S) 
    RHS = np.kron(S, I) @ B00
    equalUpToScalar(LHS, RHS)

    residual = np.kron(S, S @ np.linalg.matrix_power(Z, 2)) @ CZ

    # Relation 2
    LHS = B01 @ np.kron(I, S) 
    RHS = residual @ B01 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = B02 @ np.kron(I, S) 
    RHS = residual @ B02 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 4
    LHS = B10 @ np.kron(I, S) 
    RHS = residual @ B10 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = B11 @ np.kron(I, S) 
    RHS = residual @ B11 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = B12 @ np.kron(I, S) 
    RHS = residual @ B12 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 7
    LHS = B20 @ np.kron(I, S) 
    RHS = residual @ B20 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(I, S) 
    RHS = residual @ B21 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = B22 @ np.kron(I, S) 
    RHS = residual @ B22 @ w2I
    equalUpToScalar(LHS, RHS)

In [193]:
IxS_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [194]:
def IxX_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    residual1 = np.kron(X, Z)

    # Relation 1
    LHS = B00 @ np.kron(I, X) 
    RHS = residual1 @ B00
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(X, np.linalg.matrix_power(Z, 2))

    # Relation 2
    LHS = B01 @ np.kron(I, X) 
    RHS = residual2 @ B01
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = B02 @ np.kron(I, X) 
    RHS = np.kron(X, I) @ B02
    equalUpToScalar(LHS, RHS)

    # Relation 4
    LHS = B10 @ np.kron(I, X) 
    RHS = residual1 @ B10
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = B11 @ np.kron(I, X) 
    RHS = residual1 @ B11
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = B12 @ np.kron(I, X) 
    RHS = residual1 @ B12
    equalUpToScalar(LHS, RHS)

    # Relation 7
    LHS = B20 @ np.kron(I, X) 
    RHS = residual1 @ B20
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(I, X) 
    RHS = residual1 @ B21
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = B22 @ np.kron(I, X) 
    RHS = residual1 @ B22
    equalUpToScalar(LHS, RHS)

In [195]:
IxX_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [196]:
def IxZ_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    residual1 = np.kron(Z, I)

    # Relation 1
    LHS = B00 @ np.kron(I, Z) 
    RHS = residual1 @ B00
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(Z, np.linalg.matrix_power(Z, 2))

    # Relation 2
    LHS = B01 @ np.kron(I, Z) 
    RHS = residual2 @ B01
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = B02 @ np.kron(I, Z) 
    RHS = residual2 @ B02
    equalUpToScalar(LHS, RHS)

    # Relation 4
    LHS = B10 @ np.kron(I, Z) 
    RHS = residual2 @ B10
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = B11 @ np.kron(I, Z) 
    RHS = residual2 @ B11
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = B12 @ np.kron(I, Z) 
    RHS = residual2 @ B12
    equalUpToScalar(LHS, RHS)

    # Relation 7
    LHS = B20 @ np.kron(I, Z) 
    RHS = residual2 @ B20
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = B21 @ np.kron(I, Z) 
    RHS = residual2 @ B21
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = B22 @ np.kron(I, Z) 
    RHS = residual2 @ B22
    equalUpToScalar(LHS, RHS)

In [197]:
IxZ_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [198]:
def IxCZ_B_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(27, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(27, w2)

    residual1 = np.kron(I, np.kron(H, I) @ np.linalg.matrix_power(CZ, 2) @ np.kron(H, I)) @ np.kron(CZ, I) @ np.kron(I, np.kron(H, I) @ np.linalg.matrix_power(CZ, 2) @ np.kron(H, I)) @ np.kron(CZ, I)
    
    # Relation 1
    LHS = np.kron(B00, I) @ np.kron(I, CZ)
    RHS = residual1 @ np.kron(B00, I)
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(I, np.linalg.matrix_power(CZ, 2) @ np.kron(H, I) @ np.linalg.matrix_power(CZ, 2) @ np.kron(H, I)) @ np.kron(CZ, I) @ np.kron(I, np.kron(H, I) 
                                                                                                    @ np.linalg.matrix_power(CZ, 2) @ np.kron(H, I)) @ np.kron(CZ, I)
    # Relation 2
    LHS = np.kron(B01, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(B01, I)
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(B02, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(B02, I)
    equalUpToScalar(LHS, RHS)

    # Relation 4
    LHS = np.kron(B10, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(B10, I)
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = np.kron(B11, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(B11, I)
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = np.kron(B12, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(B12, I)
    equalUpToScalar(LHS, RHS)

    # Relation 7
    LHS = np.kron(B20, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(B20, I)
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = np.kron(B21, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(B21, I)
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = np.kron(B22, I) @ np.kron(I, CZ)
    RHS = residual2 @ np.kron(B22, I)
    equalUpToScalar(LHS, RHS)

In [199]:
IxCZ_B_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [200]:
def S_C_boxes():
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)

    # Relation 1
    LHS = C0 @ S
    RHS = S @ C0
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = C1 @ S
    RHS = Z @ S @ C1 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = C2 @ S
    RHS = np.linalg.matrix_power(Z, 2) @ S @ C2
    equalUpToScalar(LHS, RHS)

In [201]:
S_C_boxes()

Exactly equal
Exactly equal
Exactly equal


In [202]:
def Z_C_boxes():
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)

    # Relation 1
    LHS = C0 @ Z
    RHS = Z @ C0
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = C1 @ Z
    RHS = Z @ C1 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = C2 @ Z
    RHS = Z @ C2 @ wI
    equalUpToScalar(LHS, RHS)

In [203]:
Z_C_boxes()

Exactly equal
Exactly equal
Exactly equal


In [204]:
def X_C_boxes():
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)

    # Relation 1
    LHS = C0 @ X
    RHS = C1
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = C1 @ X
    RHS = C2
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = C2 @ X
    RHS = C0
    equalUpToScalar(LHS, RHS)

In [205]:
X_C_boxes()

Exactly equal
Exactly equal
Exactly equal


In [206]:
def CZ_CxI_boxes():
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)

    # Relation 1
    LHS = np.kron(C0,I) @ CZ
    RHS = CZ @ np.kron(C0,I)
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = np.kron(C1,I) @ CZ
    RHS = np.kron(I,Z @ Z) @ CZ @ np.kron(C1,I)
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = np.kron(C2,I) @ CZ
    RHS = np.kron(I,Z) @ CZ @ np.kron(C2,I)
    equalUpToScalar(LHS, RHS)

In [207]:
CZ_CxI_boxes()

Exactly equal
Exactly equal
Exactly equal


In [208]:
def SxI_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    # Relation 1
    LHS = D00 @ np.kron(S, I) 
    RHS = np.kron(I, S) @ D00
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = D01 @ np.kron(S, I) 
    RHS = np.kron(I, S) @ D01
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = D02 @ np.kron(S, I) 
    RHS = np.kron(I, S) @ D02
    equalUpToScalar(LHS, RHS)

    # Relation 4
    LHS = D10 @ np.kron(S, I) 
    RHS = np.kron(I, S) @ D10
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = D11 @ np.kron(S, I) 
    RHS = np.kron(I, S) @ D11
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = D12 @ np.kron(S, I) 
    RHS = np.kron(I, S) @ D12
    equalUpToScalar(LHS, RHS)

    # Relation 7
    LHS = D20 @ np.kron(S, I) 
    RHS = np.kron(I, S) @ D20
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = D21 @ np.kron(S, I) 
    RHS = np.kron(I, S) @ D21
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = D22 @ np.kron(S, I) 
    RHS = np.kron(I, S) @ D22
    equalUpToScalar(LHS, RHS)

In [209]:
SxI_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [210]:
def ZxI_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    # Relation 1
    LHS = D00 @ np.kron(Z, I) 
    RHS = np.kron(I, Z) @ D00
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = D01 @ np.kron(Z, I) 
    RHS = np.kron(I, Z) @ D01
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = D02 @ np.kron(Z, I) 
    RHS = np.kron(I, Z) @ D02
    equalUpToScalar(LHS, RHS)

    # Relation 4
    LHS = D10 @ np.kron(Z, I) 
    RHS = np.kron(I, Z) @ D10
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = D11 @ np.kron(Z, I) 
    RHS = np.kron(I, Z) @ D11
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = D12 @ np.kron(Z, I) 
    RHS = np.kron(I, Z) @ D12
    equalUpToScalar(LHS, RHS)

    # Relation 7
    LHS = D20 @ np.kron(Z, I) 
    RHS = np.kron(I, Z) @ D20
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = D21 @ np.kron(Z, I) 
    RHS = np.kron(I, Z) @ D21
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = D22 @ np.kron(Z, I) 
    RHS = np.kron(I, Z) @ D22
    equalUpToScalar(LHS, RHS)

In [211]:
ZxI_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [212]:
def IxX_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    # Relation 1
    LHS = D00 @ np.kron(I, X) 
    RHS = np.kron(X, I) @ D00
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = D01 @ np.kron(I, X) 
    RHS = np.kron(X, Z @ Z) @ D01
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = D02 @ np.kron(I, X) 
    RHS = np.kron(X, Z) @ D02
    equalUpToScalar(LHS, RHS)

    # Relation 4
    LHS = D10 @ np.kron(I, X) 
    RHS = np.kron(Z, I) @ D10
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = D11 @ np.kron(I, X) 
    RHS = np.kron(X @ Z, Z @ Z) @ D11 @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = D12 @ np.kron(I, X) 
    RHS = np.kron(X @ X @ Z, Z) @ D12 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 7
    LHS = D20 @ np.kron(I, X) 
    RHS = np.kron(Z @ Z, I) @ D20
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = D21 @ np.kron(I, X) 
    RHS = np.kron(X @ Z @ Z, Z @ Z) @ D21
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = D22 @ np.kron(I, X) 
    RHS = np.kron(X @ X @ Z @ Z, Z) @ D22
    equalUpToScalar(LHS, RHS)

In [213]:
IxX_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [214]:
def IxZ_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    # Relation 1
    LHS = D00 @ np.kron(I, Z) 
    RHS = np.kron(Z, I) @ D00
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = D01 @ np.kron(I, Z) 
    RHS = np.kron(Z, I) @ D01
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = D02 @ np.kron(I, Z) 
    RHS = np.kron(Z, I) @ D02
    equalUpToScalar(LHS, RHS)

    residual1 = np.kron(X @ X, Z)

    # Relation 4
    LHS = D10 @ np.kron(I, Z) 
    RHS = residual1 @ D10
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = D11 @ np.kron(I, Z) 
    RHS = residual1 @ D11
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = D12 @ np.kron(I, Z) 
    RHS = residual1 @ D12
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(X, Z @ Z)

    # Relation 7
    LHS = D20 @ np.kron(I, Z) 
    RHS = residual2 @ D20
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = D21 @ np.kron(I, Z) 
    RHS = residual2 @ D21
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = D22 @ np.kron(I, Z) 
    RHS = residual2 @ D22
    equalUpToScalar(LHS, RHS)

In [215]:
IxZ_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [216]:
def CZ_D_boxes():
    I = np.identity(3)
    w = findAllRoots(3)[0]
    wI = makeScalarId(9, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(9, w2)

    # Relation 1
    LHS = D00 @ CZ
    RHS = D02
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = D01 @ CZ
    RHS = D00
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = D02 @ CZ
    RHS = D01
    equalUpToScalar(LHS, RHS)

    residual1 = np.kron(H @ S @ np.linalg.matrix_power(H, 3), Z @ np.linalg.matrix_power(S, 2))

    # Relation 4
    LHS = D10 @ CZ
    RHS = residual1 @ D12 @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 5
    LHS = D11 @ CZ
    RHS = residual1 @ D10 @ wI
    equalUpToScalar(LHS, RHS)

    # Relation 6
    LHS = D12 @ CZ
    RHS = residual1 @ D11 @ wI
    equalUpToScalar(LHS, RHS)

    residual2 = np.kron(H @ np.linalg.matrix_power(S, 2) @ np.linalg.matrix_power(H, 3), np.linalg.matrix_power(Z, 2) @ S)

    # Relation 7
    LHS = D20 @ CZ
    RHS = residual2 @ D22 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 8
    LHS = D21 @ CZ
    RHS = residual2 @ D20 @ w2I
    equalUpToScalar(LHS, RHS)

    # Relation 9
    LHS = D22 @ CZ 
    RHS = residual2 @ D21 @ w2I
    equalUpToScalar(LHS, RHS)

In [217]:
CZ_D_boxes()

Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal
Exactly equal


In [218]:
def S_E_boxes():
    w = findAllRoots(3)[0]
    wI = makeScalarId(3, w)
    w2 = findAllRoots(3)[1]
    w2I = makeScalarId(3, w2)

    # Relation 1
    LHS = E0 @ S
    RHS = E1
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = E1 @ S
    RHS = E2
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = E2 @ S
    RHS = E0
    equalUpToScalar(LHS, RHS)

In [219]:
S_E_boxes()

Exactly equal
Exactly equal
Exactly equal


In [220]:
def Z_F_boxes():
    
    # Relation 1
    LHS = F0 @ Z
    RHS = F2
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = F1 @ Z
    RHS = F0
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = F2 @ Z
    RHS = F1
    equalUpToScalar(LHS, RHS)

In [221]:
Z_F_boxes()

Exactly equal
Exactly equal
Exactly equal


In [222]:
def Z_E_boxes():
    
    # Relation 1
    LHS = E0 @ Z
    RHS = Z @ E0
    equalUpToScalar(LHS, RHS)

    # Relation 2
    LHS = E1 @ Z
    RHS = Z @ E1
    equalUpToScalar(LHS, RHS)

    # Relation 3
    LHS = E2 @ Z
    RHS = Z @ E2
    equalUpToScalar(LHS, RHS)

In [223]:
Z_E_boxes()

Exactly equal
Exactly equal
Exactly equal
